---
# OpenFoodFacts
---
- On importe toutes les librairies nécessaire et l'on défini ses fonctions en début du notebook :)
- On défini également certaines variables utiles pour l'affichage + certaines fonctions custom qui nous seront fort utiles
- Il s'agit d'une base de données collaborative qui peut comporter des erreurs des saisie, incompréhensions, etc ..
- Afin d'alléger le notebook sur enregistrement, les graphiques sont exportés ici : http://1.x24.fr/a/jupyter/off/

<hr>

- <b>Matières exotiques</b> : éléments non listés sur les emballages des produits : ( additifs, conservateurs, eau ? )
- En regardant mon épicerie : j'ai constaté que les pates : environ 70% des 100g en nutriments : principalement des glucides, donc les 30% exotiques
- Pour la viande ( le vivant ) on déduit qu'il contient environ 70% de matière exotiques ( eau )
- Pour le quinoa et nourritures très sèches : 10% exotiques
- Pour le sucre : 100% de glucides, 0% exotiques
---


## Librairies & variables

In [ ]:
#}{Variables on top
demo=1;#1 on share, do not upload files !! print their name and links instead
sftp={'cd':'off','web':'http://1.x24.fr/a/jupyter/off/','h':'hz','u':'z','p':'z'}

acceptableScore=50;#revelancyScoring decreased according to outliers
fn='en.openfoodfacts.org.products.tsv';#MainDataframefileName
fn2=fn+'2.tsv';fn5=fn+'-cleaned.tsv'#cleanedde
fn3=fn+'3.tsv';#by country
fn4=fn+'-dummies.tsv'
allVars={}
resumed=0
lowQuantileLimit=0.001
highQuantileLimit=0.999
away=[]#les données que l'on exclut progressivement du dataframe
debug={}
outliers={}

import sys,math,pickle,matplotlib,re,os,operator,math,requests,subprocess,plotly.express as px,pandas as pd,numpy as np,matplotlib.pyplot as plt,seaborn as sns
if('webptools' not in sys.modules):
  !pip install webptools
if('pysftp' not in sys.modules):#SFTP support
  !pip install pysftp;

from scipy.ndimage.filters import gaussian_filter
from scipy import stats
from webptools import webplib
import pysftp;cnopts=pysftp.CnOpts();cnopts.hostkeys=None;sftp['cnopts']=cnopts;#fix:SSHException: No hostkey for host

pd.set_option('display.max_rows',900)
pd.set_option('display.max_columns',40)
pd.set_option('display.width',1490)#or evaluate js document frame innerWidth .
plt.style.use('fivethirtyeight')
#plt.rcParams["figure.figsize"] = (32,12)#46px par unité environ
heightPerGraph=widthPerGraph=6
plt.rcParams["figure.figsize"] = (heightPerGraph,widthPerGraph)
plt.rcParams['figure.facecolor'] = 'white'#white background for display
sns.set(style='white', font_scale=1.2)
sns.set_style('darkgrid')
#graphs 3d rotatifs ??
#%matplotlib notebook
#%matplotlib inline #

#print shorthand
def p(x):
    print(x);

#Speech Support
def say(x):
    html("<script>say(\""+x+"\")</script>");
    
def html(x):
  import IPython;
  IPython.display.display(IPython.display.HTML(x))
  return;

html("<script>lang=document.body.parentNode.getAttribute('lang');lang='en';function say(x,vid,p,rate){var defaultVoice=0,r=rate||1;if(lang.indexOf('fr')>-1){defaultVoice=2;r=1.8;}/*hortense*/vid=vid||defaultVoice;p=p||1;x=x||0;if(!x){x=alpow.getText();}if(!x)return;console.log(x);var y=new SpeechSynthesisUtterance(),voices=window.speechSynthesis.getVoices();y.voice=voices[vid];y.voiceURI='native';y.volume=1;y.rate=r;y.pitch=p;y.text=x;y.lang='en-US';speechSynthesis.speak(y);return x;}</script>")        

#ftpls() return list of files

def ftpexists(fn,cd=0):
  if(cd==0):
    cd=sftp['cd']
  liste=ftpls(cd)
  if(fn in liste):
    return True;
  return False;

def ftpls(cd=0):
  global sftp;
  if(cd==0):
    cd=sftp['cd']
  with pysftp.Connection(sftp['h'], username=sftp['u'], password=sftp['p'], cnopts=sftp['cnopts']) as connection:
    with connection.cd(cd): 
      return connection.listdir()

def ftpget(fn,cd=0):
  global sftp;
  if(cd==0):
    cd=sftp['cd']
  with pysftp.Connection(sftp['h'], username=sftp['u'], password=sftp['p'], cnopts=sftp['cnopts']) as connection:
    with connection.cd(cd): 
      if(type(fn)==str):#solo
        fn=[fn]
      for i in fn:        
        connection.get(i)
      print('get:'+','.join(fn))

def ftpput(fn,cd=0,aszip=0):
  global sftp,demo;
  if(type(fn)==str):#solo
    fn=[fn]
  if(demo):
    for i in fn:
      print(sftp['web']+i)
    return 0;#demo mode not updated

  if(cd==0):
    cd=sftp['cd']
  with pysftp.Connection(sftp['h'], username=sftp['u'], password=sftp['p'], cnopts=sftp['cnopts']) as connection:
    with connection.cd(cd):
      for i in fn:       
        #print(i.split('.')[-1]);print(os.path.getsize(i)) 
        if((i.split('.')[-1] not in ['zip','jpg','png']) & ( os.path.getsize(i)>5000000 | aszip)):
          os.system('rm '+i+'.zip')
          os.system('zip '+i+'.zip '+i)
          i=i+'.zip'
        connection.put(i)
        print('put:'+sftp['web']+i)


def getFile(fn,sep='\t'):
  if(os.path.exists(fn)==True):
    return 1;

  elif(os.path.exists(fn+'.zip')==True):  
    os.system('unzip '+fn+'.zip')#should have been unzipped ...
    return 2;

  elif(ftpexists(fn)==True):
    ftpget(fn);
    return 3;

  elif(ftpexists(fn+'.zip')==True):
    ftpget(fn+'.zip');
    os.system('unzip '+fn+'.zip')#unzip
    return 4;

  print('#Fatal:'+fn+' not found')
  return 0;
  assert(False)
  return 0;

def loadCsvOrZipOrFTPOrDrive(fn,sep='\t'):
  if(getFile(fn)):
   return pd.read_csv(fn, sep='\t');
  return 0;  
#df.shape[0]

def nullValues(z):
    col_mask=z.isnull().any(axis=0);row_mask=z.isnull().any(axis=1);return z.loc[row_mask,col_mask];

def asort(dict):
    if type(dict)==list:
        return dict;#allready
    return sorted(dict.items(),key=operator.itemgetter(1),reverse=False)

def arsort(dict):
    if type(dict)==list:
        return dict;#allready
    return sorted(dict.items(),key=operator.itemgetter(1),reverse=True)

def filename(x):
    return re.sub(r"[^a-z0-9\.\-_:]+",'', x.lower());

def uniqueValuesPerColumn(sc,df2):
  return df2.groupby(sc)[sc].dagg(['count']).sort_values(by='count',ascending=False)

def dfQuery(df2,sc,query,debug=0):
  x=df2[df2[sc].astype(str)==query];#exact match
  if(len(x)):
    return x;
  x=df2[df2[sc].astype(str).str.contains(query,na=False,case=False)]
  if(len(x)):
    return x;
  if debug:
    print('no results for '+sc+' : '+query)
  return pd.DataFrame(columns=list(df2.columns.values));#empty dataframe  

def ftpputzip(fn,cd=0):
  if(type(fn)==str):#solo
    fn=[fn]
  zipped=[]
  for i in fn:
    os.system('rm '+i+'.zip');
    os.system('zip '+i+'.zip '+i);
    zipped.append(i+'.zip')
  ftpput(zipped,cd)  

#as pickle
def fgc(fn):
  fn=fn+'.pickle'
  getFile(fn)
  data=open(fn,"rb");ret=pickle.load(data);data.close();return ret;

def FPC(fn,data):
  fn=fn+'.pickle'
  f=open(fn,"wb");f.write(pickle.dumps(data));f.close();
  os.system('rm '+fn+'.zip');#rm potential zip file before upload otherwise will upload previous version
  os.system('zip '+fn+'.zip '+fn);
  ftpput(fn+'.zip')  

def sg(fn):#shortGet dataframe
  !pip install pysftp;
  import pysftp,pandas as pd;cnopts=pysftp.CnOpts();cnopts.hostkeys=None;#fix:SSHException: No hostkey for host
  sftp={'cd':'off','web':'---jupyter','h':'-','u':'-','p':'-','cnopts':cnopts}
  with pysftp.Connection(sftp['h'], username=sftp['u'], password=sftp['p'], cnopts=sftp['cnopts']) as connection:
    with connection.cd(sftp['cd']): 
      connection.get(fn+'.zip')
  os.system('unzip '+fn+'.zip')
  return pd.read_csv(fn,sep='\t');

def lastCommaValue(x):
  if(type(x)!=str):
    return x
  if(x.find(',')<0):
    return x.lower();
  return x.split(',')[-1].lower()

#unify some categories
def lowerNoSpace(x):
  if(type(x)!=str):
    return x
  return re.sub(r"[^a-z0-9:]+",'', x.lower());

#}{Graphs
def pltinit(df,i,j,axis=0):
  x=df[i];y=df[j];
  if(axis!=0):
    return [x,y,0,0,axis];    
  fig,ax=plt.subplots()  
  plt.title(i+' vs '+j);plt.xlabel(i);plt.ylabel(j);
  return [x,y,filename(i+'.'+j),fig,ax];
    
def plot(df,i,j):
  x,y,fn,fig,ax=pltinit(df,i,j)
  fn='plot'+fn+'.png'
  #bestCorrelationsKeys.keys():
  plt.plot(x,y)  
  plt.savefig(fn, bbox_inches = 'tight')
  webp(fn)
  show()
  return fn

#bins: nb groupe de valeurs, trop et on ne vera plus rien
def myplot(x, y, s, bins=100):
  heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins)
  heatmap = gaussian_filter(heatmap, sigma=s)
  extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
  return heatmap.T, extent

#VMAX=log Max value for heatmap,2 plus lisse plus joli à voir que 1
def heat(df,a='',b='',s=1,vmin=0,vmax=100,bins=100,axis=0,xscale=0,yscale=0,fn=0):
  if(fn==0):
    fn='heat.'+filename(a+'.'+b)+'.png'; 

  #recombiné avec son index  
  if(type(df)==pd.core.series.Series):
    if(len(df)==0):
      print('empty.. skipping')
      return 0;     
    img,extent=myplot(range(df.shape[0]),df,s,bins=bins)              
    #img,extent=myplot(df.index,df,s,bins=bins)        

    if(axis):
      axis.imshow(img, extent=extent, origin='lower', cmap=matplotlib.cm.jet, vmax=vmax, vmin=vmin, aspect="auto")
      #axis.set_title(fn)
      axis.set_xlabel(a)
      axis.set_ylabel(b)
      if(yscale):
        axis.set_yscale(yscale)
      if(xscale):
        axis.set_xscale(xscale)
      return 1;
    
    plt.imshow(img, extent=extent, origin='lower', cmap=matplotlib.cm.jet,vmax=vmax,vmin=vmin, aspect="auto")
    #plt.title(fn)
    plt.xlabel(a)
    plt.ylabel(b)
    if(yscale):
      plt.yscale(yscale)
    if(xscale):
      plt.xscale(xscale)
    plt.savefig(fn)
    ftpput(fn) 
    show()
    return 1;
### Regular Dataframe
  x=df[a];y=df[b];
  if(bins==0):#set to auto
    bins=[x.max()-x.min(),y.max()-x.min()];
    print(bins)

  img, extent = myplot(x,y,s,bins=bins)  
  #Ratio entre les deux ??
  if(axis!=0):
    #print(axis)
    axis.set_facecolor('white')
    axis.set_xlabel(a);axis.set_ylabel(b);
    #imshow TypeError: inner() got multiple values for argument 'ax'
    axis.imshow(img, extent=extent, origin='lower', cmap=matplotlib.cm.jet, vmax=vmax, vmin=vmin, aspect="auto")
    #axis.set_title(b+' / '+a)
  else:
    #plt.title(b+' / '+a)
    plt.figure().set_facecolor('white')
    plt.imshow(img, extent=extent, origin='lower', cmap=matplotlib.cm.jet,vmax=vmax,vmin=vmin, aspect="auto")
    plt.savefig(fn)
    ftpput(fn)
    show()
#scatter(sup0,i,'bigdots',size=20,opacity=1,fn='bigdots.png')      

#out of bounds values <<< d'après kde2
def getLimit(i):
  return limitFor(i)
  
def limitFor(i):
  limit=100
  if('carbon' in i):
    return 999999999;#footprint
  if('fiber' in i):
    return 52;#canelle
  if('energy' in i):    
    return 3700;#huile 100g considéré comme 100ml, les gens ne font pas vraiment la différence : 4093  
  if('cholesterol' in i):
    return 3.1;#cervelle de veau
  if('calcium' in i):
    return 2;#	meloukhia en poudre ??
  if('iron' in i):
    return 0.2;
#A:beta catorène:30mg foie de morue, 200mg C pour Cassis    
  if('vitamin-a' in i):
    return 0.03;
  if('vitamin-c' in i):
    return 0.2;
  if('sodium' in i):
    return 40;
  if('nutrition-score' in i):
    return 40;
  return limit
  
#https://sante.journaldesfemmes.fr/calories/classement/aliments/potassium        
#}{IO,MIOS,Rechargement des données
#DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False. if self.run_code(code, result):
#pd.read_csv DtypeWarning Columns have mixed types low_memory=False    
def exportDf(df,fn, sep='\t'):
  os.system('rm '+fn+' '+fn+'.zip');#rm potential zip file before upload otherwise will upload previous version
  df.to_csv(fn, sep=sep,index=False)  
  os.system('zip '+fn+'.zip '+fn);
  ftpput(fn+'.zip')

#df=loadData(fn4)
def loadData(x,sep='\t'):    
  getFile(x)
  global tot,rows,debug;
  df = pd.read_csv(x, sep=sep);
  #df=reduce_mem_usage(df)#not doing anythin..
  rows=df.shape[0]
  tot=df.size
  debug={'total cells':tot};
  say('data loaded');df[:1]
  print(df.info());#mixed columns values type might mess up later
  print('_'*140)
  print(df.describe());#SneakPeak  
  print('_'*140)
  return df  

def cleanData(dforiginal,fillWithMeanValue=1): 
  df=dforiginal.copy(deep=True)
  #test = pd.read_csv('../input/test.csv')
  null_columns=df.columns[df.isnull().any()]
  print('Total Row*Columns in dataset : '+str(df.size));#Diagnose null columns
  print('Total null values : '+str(nullValues(df).size));#Diagnose null columns
  # Fournissent une bonne indication des colonnes à dropper pour le modèke

  print('_'*140)
  df.replace([np.inf,-np.inf],np.nan,inplace=True)#replace infinite by Nan
  print(df[null_columns].isnull().sum().sort_values(ascending=False))
  if(fillWithMeanValue):
    df.fillna(df.mean(),inplace=True)#Numeric types corrected
  df.fillna(0,inplace=True)#2 rows of Nan -> is valid either for strings or numeric datatype
  print('_'*140)
  print('Null values then: '+str(nullValues(df).size))#CAUTION !!! WONT FIT INTO MODEL OTHERWISE
  print('_'*140)
  return df;      

# Pour grouper sur des chaines de caractères => les unifier tant que l'on peut ..
def lastCommaValue(x):
  if(type(x)!=str):
    return x
  if(x.find(',')<0):
    return x.lower();
  return x.split(',')[-1].lower()

#unify some categories
def lowerNoSpace(x):
  if(type(x)!=str):
    return x
  return re.sub(r"[^a-z0-9:]+",'', x.lower());

#https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65,https://www.kaggle.com/c/champs-scalar-coupling/discussion/96655
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max and c_prec == np.finfo(np.float16).precision:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

#type(sup0)==pandas.core.series.Series
def scatter(df,a='',b='',ts=0,minx=0,maxx=0,miny=0,maxy=0,opacity=0.02,color='blue',size=2,axis=0,xscale=0,yscale=0,reg=0,fn=0):
  if(fn==0):
    fn=filename(a+'.'+b+'.png'); 
  #recombiné avec son index  
  if(type(df)==pd.core.series.Series):#{
    if(len(df)==0):
      print('empty.. skipping')
      return 0;   
    #x=df.index
    #print('serie')
    if(maxy==0):#maxx or de propos
      maxy=df.max()  
    x=range(df.shape[0])  

    if(axis):
      axis.scatter(x,df,s=size,alpha=opacity)
      #axis.set_title(fn)
      #  ax.set_xlim(minx,maxx);ax.set_ylim(miny,maxy)
      axis.set_xlabel(a)
      axis.set_ylabel(b)
      axis.set_facecolor('white')
      if(yscale):
        axis.set_yscale(yscale)
      if(xscale):
        axis.set_xscale(xscale)
      return 1;
    
    plt.figure().set_facecolor('white')
    plt.scatter(x,df,s=size,alpha=opacity)
    #plt.title(fn)
    if maxy:
      plt.ylim(miny,maxy)      
    plt.xlabel(a)
    plt.ylabel(b)
    if(yscale):
      plt.yscale(yscale)
    if(xscale):
      plt.xscale(xscale)
    plt.savefig(fn, bbox_inches='tight')
    #ftpput(fn) 
    webp(fn)
    show()
    return fn;
#}end pandas serie    
### Sinon Dataframe ordinaire
  elif(axis==0):
    #print('joinplot')
    x=df[a];y=df[b]
    args={'x':x,'y':y, 'data':df, 'joint_kws': {'alpha':opacity,'color':color}}
    if(reg):
      args['kind']='reg';      
      args['joint_kws']={'color':color};     
      args['scatter_kws']={'alpha':opacity};     
      
    #print(args);#TypeError: regplot() got an unexpected keyword argument 'alpha'
    sns.jointplot(**args).savefig(fn, bbox_inches='tight');
    webp(fn)
    show()
    return fn;
### Rendu sur un axe ..
  x,y,fn2,fig,ax=pltinit(df,a,b,axis)

  if(maxx==0):
    maxx=x.max()  
  if(maxy!=0): 
    maxy=y.max()

  if(axis!=0):#specifié non généré
    axis.set_xlabel(a);axis.set_ylabel(b);
    axis.scatter(x,y,alpha=opacity,c=color,s=size);
    ax=axis

  ax.set_facecolor('white')

  if(xscale):
    ax.set_xscale(xscale)
  if(yscale):
    ax.set_yscale(yscale)
  return 1
  
  if(ts & False):
    plt.gca().set_yticklabels(backgroundcolor='white',labels=y,rotation=(0),fontsize=ts,linespacing=ts)     
    #ax.tick_params(axis='y',which='major',pad=ts)
    #ax.set_yticklabels(labels=y,rotation = (45), fontsize = 10, va='bottom', ha='left')      
  if(axis==0):    
    if False:    
      plt.scatter(x,y,alpha=opacity,c=color,s=size);    
      plt.show();
      plt.savefig(fn, bbox_inches='tight')    
      webp(fn)

def r2(a,b):
  return stats.pearsonr(a,b)[0] ** 2
#snscat
def snsscat(df,a='',b='',ts=0,minx=0,maxx=0,miny=0,maxy=0,opacity=0.02,color='blue',size=2,axis=0,xscale=0,yscale=0,fn=0,kind='scatter'):
  if(fn==0):
    fn=filename(a+'.'+b+'.png'); 
  x=df[a];y=df[b];#fig,ax=plt.subplots(1);#ax=ax.flatten()
  slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
  corr=stats.pearsonr(x, y)[0] ** 2
  #x,y,fn2,fig,ax=pltinit(df,a,b,axis)
  #ax.scatter(x,y,alpha=opacity,c=color,s=size);
  if(maxx==0):
    maxx=x.max()  
  if(maxy==0): 
    maxy=y.max()
#https://stackoverflow.com/questions/36191906/rescale-axis-on-seaborn-jointgrid-kde-marginal-plots    'ax':ax, 'height':height,
#https://seaborn.pydata.org/generated/seaborn.JointGrid.html
  args={'dropna':True,'kind':kind,'x':x,'y':y, 'data':df, 'joint_kws': {'alpha':opacity,'color':color},'xlim':[minx,maxx],'ylim':[miny,maxy],'color':'red'} #
  args['stat_func']=r2
  args['marginal_kws']=dict(bins=15, rug=True)
  args['annot_kws']=dict(stat="r");      #joinplot annot_kws
#N'apparait pas ..

  if(kind in ['reg']):
    args['joint_kws']={'color':color}#reg no opacity
    args['line_kws']={'label':"y={0:.1f}x+{1:.1f}".format(slope,intercept)};#not hex,nor kde  
    args['scatter_kws']={'alpha':opacity}; #,nor kde
  if(kind in ['kde']):    
    args['joint_kws']={'color':color}
    args['marginal_kws']={}#no bins property
    pass;
#print(args);#TypeError: regplot() got an unexpected keyword argument 'alpha',dir(ax)
  g=sns.jointplot(**args)  
  #g=sns.JointGrid(**args)  
  """
  g = sns.JointGrid(x="total_bill", y="tip", data=tips)
  g = g.plot_joint(plt.scatter,color="g", s=40, edgecolor="white")
  g = g.plot_marginals(sns.distplot, kde=False, color="g")
  g = g.annotate(stats.pearsonr)

  g = g.plot_joint(sns.kdeplot, cmap="Blues_d")
  g = g.plot_marginals(sns.kdeplot, shade=True)
  """
  if(xscale):
    g.set_xscale(xscale)
  if(yscale):
    g.set_yscale(yscale)
  #ax.set_xlabel(a);ax.set_ylabel(b);#déjà attribués
#AttributeError: 'JointGrid' object has no attribute 'set_xlim'     
  if((kind in ['reg','kde']) & False):      
    regline=g.ax_joint.get_lines()[0]
    regline.set_color('red')
    regline.set_zorder('5')
  #fig.text(1, 1, "An annotation", horizontalalignment='left', size='medium', color='black', weight='semibold')
  #dir(g)
  #rsquare = lambda a, b: 
  #rsquare = lambda a, b: 2;g.annotate(rsquare)
  #ax = g.axes[0,0]
  plt.title(str(int(slope))+'-'+str(int(intercept)))
  g.savefig(fn, bbox_inches='tight');
  webp(fn)
  #ftpput(fn)     
  show()
  #plt.show()
  return 1;

def show():
  global demo
  if(demo):
    plt.show();
  else:
    plt.close()

def webp(x):
  x2=x.replace('.png','')+'.webp'
  webplib.cwebp(x,x2, "-q 70")
  ftpput(x2)

#https://stackoverflow.com/questions/18090672/convert-dictionary-entries-into-variables-python
def restore():
  resume();
  
def resume():
  global resumed,allVars#,df,valeursExotiques,strings1,strings2,Cols100g,outliers,debug
  if(resumed):
    print('allready resumed ..')
    return;
  resumed=1
  getFile('allVars')#allVars.keys()
  allVars=fgc('allVars')  
  globals().update(allVars)#locals() for function only
#resume()#pour aller Fast Forward sur d'autres cellules de code

#print(webp.cwebp("1.png", "1.webp", "-q 70"));ftpput('1.webp')
def radar(df,seuilSuppression=1):
  sel=df.copy(deep=True)
  #normalization valeur numérique max à 100 !!
  if('energy_100g' in sel.keys()):
    sel['energy_100g']=sel['energy_100g']/370
  #print(sel);print(sel['product_name'].values[0]);print(sel['code'].values[0])
  title=sel['product_name'].values[0]+' - '+str(sel['code'].values[0])
  rm=[]
  for i in sel.keys():
    #print(i);print(sel[i].values[0])
    #print(type(sel[i].values[0]))
    if(i in 'code,product_name,relevancy,created_t,last_modified_t'.split(',')):
      rm.append(i)
    elif(type(sel[i].values[0])==str):
      rm.append(i)#non pertinentes
    elif((sel[i].values[0]==0)|(np.isnan(sel[i].values[0]))):#
      rm.append(i)
    elif(sel[i].values[0]<seuilSuppression):
      rm.append(i)
    else:
      pass;#print(i);print(str(type(sel[i].values[0])));print(sel[i].values[0])
      #print('del:'+i      
  #print(rm)
  sel=sel.drop(rm,axis=1)
  #print(sel.keys())
#Exception: Data must be 1-dimensional
  radardf = pd.DataFrame(dict(  r=sel.values[0], theta=sel.keys()))
  fig = px.line_polar(radardf, r='r', theta='theta', line_close=True,title=title)
  fig.update_traces(fill='toself')
  fig.show()    

print('Current space usage')#Total session space usage
!du -sh . 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



     |████████████████████████████████| 18.1MB 1.3MB/s 
     |████████████████████████████████| 215kB 3.9MB/s 
     |████████████████████████████████| 768kB 8.5MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 2.7MB 16.2MB/s 
  Created wheel for pysftp: filename=pysftp-0.2.9-cp36-none-any.whl size=15483 sha256=baf4c5c061cd375c94099033d635293bcca42d2f65580068fd2a1e540e054fb1
  Stored in directory: /root/.cache/pip/wheels/d4/53/02/181ea4f6c795b40c4f1d2022ba2ed15cf811d22975eb2e5968
Successfully built pysftp


/usr/local/lib/python3.6/dist-packages/pysftp/__init__.py:61: UserWarning:

Failed to load HostKeys from /root/.ssh/known_hosts.  You will need to explicitly load HostKeys (cnopts.hostkeys.load(filename)) or disableHostKey checking (cnopts.hostkeys = None).



Current space usage
55M	.


### Load
- Fonctions I/O de rechargement des données
Dans laquelle on lit le dataframe
- applique 2 nouvelles colonnes de scoring
- ainsi que l'initialisation de certaines variables globales
- la fonction sert si l'on corrompt son dataFrame dans son analyse exploratoire ( erreur de débutant déjà commise .. )
- le dict debug est là pour conserver des stats .. 
- évidemment cela charge la RAM en conséquence du nombre de lignes totales

In [ ]:
getFile(fn)
if(os.path.exists(fn)!=True):
  assert(False)

df=loadData(fn);
df.insert(0,"relevancy",100) 
df.insert(1,"notes",'') 

get:en.openfoodfacts.org.products.tsv.zip


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning:

Columns (0,3,5,19,20,24,25,26,27,28,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356027 entries, 0 to 356026
Columns: 163 entries, code to water-hardness_100g
dtypes: float64(107), object(56)
memory usage: 442.8+ MB
None
____________________________________________________________________________________________________________________________________________
       no_nutriments    additives_n  ingredients_from_palm_oil_n  ingredients_from_palm_oil  ingredients_that_may_be_from_palm_oil_n  ingredients_that_may_be_from_palm_oil  nutrition_grade_uk    energy_100g  energy-from-fat_100g      fat_100g  saturated-fat_100g  -butyric-acid_100g  -caproic-acid_100g  -caprylic-acid_100g  -capric-acid_100g  -lauric-acid_100g  -myristic-acid_100g  -palmitic-acid_100g  -stearic-acid_100g  -arachidic-acid_100g  ...  copper_100g  manganese_100g  fluoride_100g  selenium_100g  chromium_100g  molybdenum_100g  iodine_100g  caffeine_100g  taurine_100g    ph_100g  fruits-vegetables-nuts_100g  fruits-vegetables-nuts-estimate_100g  collag

## Exploration
- Tout d'abord on se donne les moyens de le faire correctement, ici en imprimant la liste des colonnes séparées par une tabulation
- Il s'agit d'une base de données collaborative ..
- Il y a donc 356026 lignes, avec des infos réparties entre 165 colonnes pour un usage Ram de 448Mo
- Les colonnes max et mean sont très utiles afin de repérer des valeurs Abérrantes ( nommées "out of bounds" ou outliers ), on constate rapidement que energy_100g possède des valeurs extravageantes, et que la moyenne nous indique une forte chance que cela soit en kJ, et non en kCal ..
- Même constat pour les colonnes _100g, il ne peut y avoir plus 100g d'un élément dans un échantillon de 100g .. de même que la sommes des composantes _100g ne peut excéder 100g

<hr>

- <b>L'exploration se réalise en réalité progressivement, au fur à mesure du nettoyage / exploitation de données, nous allons ainsi tenter de regrouper les informations utiles ci-dessous</B>


### Describe
- On augmente l'affichage du nb de colonnes afin de toutes pouvoir les visualiser
- Cette commande nous retourne le nb de valeurs, leurs moyenne, leur écart std, leur valeurs minimales, maximales, ainsi que leur déciles ( 25,50,75 ) qui permettent d'identifier rapidement de potentielles valeurs abérrantes

In [ ]:
pd.set_option('display.max_columns',df.shape[1])
df.describe()

,relevancy,no_nutriments,additives_n,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,nutrition_grade_uk,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
count,356027.0,0.0,283867.000000,283867.000000,0.0,283867.000000,0.0,0.0,295367.000000,869.000000,2.794970e+05,263823.000000,0.0,0.0,1.0,2.000000,4.000000,1.0,1.0,1.0,27.000000,26.000000,0.0,0.0,1.0,0.0,23040.000000,23075.000000,938.000000,201.000000,43.000000,89.000000,207.000000,159.000000,9.000000,27.000000,26.000000,25.000000,15.000000,0.0,1.600000e+01,0.0,0.0,0.0,143443.000000,144170.000000,2.792200e+05,279186.000000,85.000000,32.000000,44.000000,278.000000,6.00000,11.000000,291.000000,451.000000,2.206830e+05,2.941610e+05,29.000000,17.000000,9.000000,289739.000000,289694.000000,4577.000000,137706.000000,35.000000,7207.000000,1472.000000,930.000000,141060.000000,11277.000000,10925.000000,11831.000000,6897.000000,5330.000000,3046.000000,5.390000e+03,331.000000,2551.000000,73.000000,99.000000,24848.000000,198.000000,141517.000000,5938.000000,140660.000000,6392.000000,3964.000000,2122.000000,1639.000000,99.000000,1181.000000,23.000000,20.000000,288.000000,91.000000,33.000000,52.000000,3228.000000,404.000000,182.000000,1383.000000,0.0,278.000000,254856.000000,254856.000000,0.0,0.0
mean,100.0,NaN,1.876851,0.023430,NaN,0.059736,NaN,NaN,1125.453320,587.216617,5.606587e+04,5.092460,NaN,NaN,7.4,6.040000,36.136182,18.9,8.1,3.0,10.383852,10.265385,NaN,NaN,61.0,NaN,10.501218,6.372548,3.476176,2.200218,2.805791,1.581979,18.734295,12.463013,0.051344,0.158230,0.063246,41.243200,26.053333,NaN,1.343750e-06,NaN,NaN,NaN,0.073487,0.020065,5.614020e+04,15.673428,12.822353,2.864063,24.836364,6.007263,17.10000,12.163636,30.943230,49.098894,3.843467e+05,5.326598e+04,4.398966,2.400000,0.021678,1.941281,0.764401,8.278337,0.000398,0.503896,0.000008,0.053610,0.033854,0.023394,0.332167,0.256490,0.020410,0.046952,0.006820,0.205586,2.741797e-04,0.000134,0.069210,0.433009,0.132496,0.424284,0.079895,0.127284,0.613267,0.003791,0.531415,0.007921,0.025613,0.003165,0.010042,0.003098,0.001478,0.000230,0.000465,2.075503,12.262055,6.476138,33.392680,60.360124,15.362637,52.102675,NaN,335.790664,9.166137,8.980656,NaN,NaN
std,0.0,NaN,2.501022,0.153094,NaN,0.280660,NaN,

### Colonnes
- Utilité de les recopier sur un bloc notes pour les prochains traitements

In [ ]:
columns=df.columns
','.join(columns)

'relevancy,notes,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturat

### Portion de données

In [ ]:
df.iloc[300:302]

,relevancy,notes,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,...,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
300,100,,433821494,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489133905,2017-03-10T08:18:25Z,1489133905,2017-03-10T08:18:25Z,Oatmeal Raisin Cookies,NaN,NaN,NaN,NaN,Bart & Judy's,bart-judy-s,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,17.0,NaN,NaN
301,100,,433821500,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489133905,2017-03-10T08:18:25Z,1489133905,2017-03-10T08:18:25Z,The Best Peanut Butter Cookies,NaN,NaN,NaN,NaN,Bart & Judy's,bart-judy-s,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,18.0,NaN,NaN


#### Examples de filtre de recherche
- Rapide à partir de valeurs maximales >> Afin de "déblayer" les anomalies, évaluer la pertinence générale de ces dernières, et faire ses premières suppositions

In [ ]:
#On constate que certaines valeurs max pour 100g excèdent les 100g
df.sort_values(by='sugars_100g',ascending=False)[['countries_en','product_name','energy_100g','sugars_100g','proteins_100g','fat_100g','salt_100g']]

#'alcohol_100g' not in index .. :()
#Va ressortir très probablement de l'huile ou alcohol à 100%  ( 1g alcool = 7kcal ou 29kj, un peu moins que pour l'essence ( 44,8kJ par MilliLitre ) ) ref : https://fr.wikipedia.org/wiki/Pouvoir_calorifique
df[(df['countries_en'] == 'Russia')].sort_values(by='energy_100g',ascending=False)[['product_name','energy_100g','sugars_100g','proteins_100g','fat_100g','salt_100g']][:10]

### Analyse d'un record précis
- Il convient également d'avoir sous la main une fonction d'interrogation rapide de ce dernier, ce qui aide grandement à la compréhension des variables ..

In [ ]:
x=dfQuery(df,'code','16193')
print(x)#or index:197147
radar(x,2)#x[Cols100g],spyder ou étoile ou coordonnées polaires

   relevancy notes   code                                                url          creator   created_t      created_datetime last_modified_t last_modified_datetime                  product_name generic_name quantity packaging packaging_tags          brands     brands_tags categories categories_tags categories_en origins  ... copper_100g manganese_100g fluoride_100g selenium_100g chromium_100g molybdenum_100g iodine_100g caffeine_100g taurine_100g ph_100g fruits-vegetables-nuts_100g fruits-vegetables-nuts-estimate_100g collagen-meat-protein-ratio_100g cocoa_100g chlorophyl_100g carbon-footprint_100g nutrition-score-fr_100g nutrition-score-uk_100g glycemic-index_100g water-hardness_100g
8        100        16193  http://world-en.openfoodfacts.org/product/0000...  usda-ndb-import  1489055651  2017-03-09T10:34:11Z      1489055651   2017-03-09T10:34:11Z  Organic Dark Chocolate Minis          NaN      NaN       NaN            NaN  Equal Exchange  equal-exchange        NaN             NaN 

### Quelles colonnes ne pas supprimer ? > Catégories
- Distingue les valeurs numériques et les valeurs textuelles ( dont les outliers sont plus difficiles à identifier car non identifiables sur un graph )
- Permet d'identifier les doublons potentiels, si absence de clé unique, on peut créer une clé composite de plusieurs renseignements authentiques d'un produit


In [ ]:
#categories_en,main_category_en,pnns_groups_2,pnns_groups_1,nutrition_grade_fr,stores                 
for i in columns:#str(df4.dtypes['code'])  #df.dtypes
  if(str(df.dtypes[i])=='object'):
    print('_'*140)
    print(uniqueValuesPerColumn(i,df)[:3])
  else:
    print(df[i].describe())

____________________________________________________________________________________________________________________________________________
        count
notes        
       356027
____________________________________________________________________________________________________________________________________________
             count
code              
70650800367      3
16571910303      2
20004552         2
____________________________________________________________________________________________________________________________________________
                                                    count
url                                                      
http://world-en.openfoodfacts.org/product/00000...      1
http://world-en.openfoodfacts.org/product/33025...      1
http://world-en.openfoodfacts.org/product/33025...      1
____________________________________________________________________________________________________________________________________________
         

#### GroupBy Pie ( Camembers )
- Appropriée pour les chaines 'strings'

In [ ]:
cols='countries_en,brands,categories,product_name,creator,pnns_groups_1,pnns_groups_2,traces,allergens,stores,purchase_places,manufacturing_places,origins_tags'.split(',')
#f,ax=plt.subplots(1,2,figsize=(32,6));j=-1;
plt.rcParams["figure.figsize"] = (36,12)
for i in cols:    
    fig, ax = plt.subplots(facecolor='white')
    x=df.groupby(i).agg({'relevancy':'count'}).sort_values(by='relevancy',ascending=False)[:20]
# Specify background color for the axis/plot,Be sure to specify facecolor or it won't look right in Illustrator : ax.set_axis_bgcolor("lightslategray");ax.set_clip_on(False)
    x.plot(kind='pie',subplots=True, ax=ax);plt.title('Répartition selon '+i);
    x="30-camembert-"+i+".png"
    fig.savefig(x,facecolor='white',transparent=False, bbox_inches = 'tight')
    webp(x)
    #ftpput(x)
    show()

put:http://1.x24.fr/a/jupyter/off/30-camembert-countries_en.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-brands.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-categories.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-product_name.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-creator.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-pnns_groups_1.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-pnns_groups_2.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-traces.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-allergens.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-stores.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-purchase_places.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-manufacturing_places.webp
put:http://1.x24.fr/a/jupyter/off/30-camembert-origins_tags.webp


## Pré-Nettoyage

### Doublons 
- Vérification par code ( colonne ean : la plus susceptible d'être unique et non inventée )

In [ ]:
print(uniqueValuesPerColumn('code',df))

              count
code               
70650800367       3
16571910303       2
20004552          2
42187216195       2
74880030037       2
...             ...
91475630069       1
91475630045       1
91475407999       1
91475159300       1
999990026839      1

[355839 rows x 1 columns]


### Suppression des doublons par code

In [ ]:
c='code'
df.drop_duplicates(subset=c,keep=False,inplace=True) 
deleted=rows-df.shape[0]
print(str(deleted)+' doublons par '+c+' effacés')
rows=df.shape[0]

349 doublons par code effacés


### Valeurs vides : >99.8% 2 pour mille
- Considérons qu'une colonne ayant un taux de remplissage inférieur à 2%% est candidate à sa propre suppression
- On souhaite toutefois conserver certaines valeurs "exotiques" afin d'établir de futures potentielles corrélations

In [ ]:
#df.describe().loc['count'].sort_values(ascending=True)[:30]#Nb de valeurs remplies par colonnes
seuilDeRemplissageValide=df.shape[0]*0.2;#2 pour Mille
serie=df.describe().loc['count']
#emptyValues=df[(df[i]==np.NaN) | (df[i]==0) | (df[i].isnull())].shape[0]#349455
valeursExotiquesColonnes=serie[serie < seuilDeRemplissageValide].sort_values(ascending=False)
valeursExotiquesColonnes=list(valeursExotiquesColonnes.index)
valeursExotiques=df[valeursExotiquesColonnes]#Lots Of NaN
x='valeursExotiques.tsv';away.append(x)
exportDf(valeursExotiques,x)
allVars['valeursExotiques']=valeursExotiques
df=df.drop(valeursExotiquesColonnes,axis=1)
print(','.join(valeursExotiquesColonnes))
#Les données dont nous n'avons pas vraiment besoin ..
if False:
  emptyColumns='water-hardness_100g,nutrition_grade_uk,-cerotic-acid_100g,-caproic-acid_100g,-melissic-acid_100g,glycemic-index_100g,-elaidic-acid_100g,-mead-acid_100g,-butyric-acid_100g,ingredients_that_may_be_from_palm_oil,chlorophyl_100g,-nervonic-acid_100g,-erucic-acid_100g,no_nutriments,-lignoceric-acid_100g,ingredients_from_palm_oil'.split(',')
  useless='url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,image_small_url,image_url'.split(',');#'code',
  x='-arachidic-acid_100g,allergens,labels,labels_tags,labels_en,additives_en,additives_tags,additives_n,ingredients_from_palm_oil_n,ingredients_from_palm_oil_tags,traces,traces_tags,traces_en'.split(',');
  df=df.drop(emptyColumns+useless+x,axis=1);


put:http://1.x24.fr/a/jupyter/off/valeursExotiques.tsv.zip
potassium_100g,polyunsaturated-fat_100g,monounsaturated-fat_100g,vitamin-pp_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-d_100g,vitamin-b6_100g,magnesium_100g,phosphorus_100g,vitamin-b12_100g,vitamin-b9_100g,alcohol_100g,zinc_100g,fruits-vegetables-nuts_100g,folates_100g,pantothenic-acid_100g,copper_100g,manganese_100g,vitamin-e_100g,cocoa_100g,selenium_100g,omega-3-fat_100g,vitamin-k_100g,energy-from-fat_100g,polyols_100g,fruits-vegetables-nuts-estimate_100g,biotin_100g,starch_100g,iodine_100g,-lactose_100g,carbon-footprint_100g,omega-6-fat_100g,-alpha-linolenic-acid_100g,chloride_100g,collagen-meat-protein-ratio_100g,-linoleic-acid_100g,fluoride_100g,bicarbonate_100g,caffeine_100g,-sucrose_100g,-docosahexaenoic-acid_100g,ph_100g,silica_100g,-fructose_100g,-eicosapentaenoic-acid_100g,beta-carotene_100g,taurine_100g,-glucose_100g,casein_100g,omega-9-fat_100g,chromium_100g,molybdenum_100g,serum-proteins_100g,-oleic-acid_100g,-ma

### Valeurs "strings" uniques non pertinentes pour corrélations .. 
- Il convient de supprimer les colonnes ayant peu de valeurs communes : ex : date, url, image, ainsi que celles remplies de valeurs textuelles séparées par des virgules, ainsi que les colonnes groupées dont la plus forte occurence représente moins de 1% du Dataframe..

In [ ]:
strings1Colonnes='image_url,image_small_url,last_modified_datetime,last_modified_t,created_datetime,created_t,url'.split(',')#code
#','.join(df.columns)
strings1=df[strings1Colonnes]#Lots Of NaN
x='strings1.tsv';away.append(x)
exportDf(strings1,x)
allVars['strings1']=strings1
df=df.drop(strings1Colonnes,axis=1)  
print(','.join(strings1Colonnes))

colonnesStrings=df.select_dtypes(include=['object']).columns
for i in colonnesStrings:
  print('_'*120)
  print('unique non null values for : '+i)
  print(df[df[i]!=0].groupby(i)[i].count().sort_values(ascending=False).head(10))

put:http://1.x24.fr/a/jupyter/off/strings1.tsv.zip
image_url,image_small_url,last_modified_datetime,last_modified_t,created_datetime,created_t,url
________________________________________________________________________________________________________________________
unique non null values for : notes
notes
    355678
Name: notes, dtype: int64
________________________________________________________________________________________________________________________
unique non null values for : code
code
999990026839    1
91475850009     1
91475041896     1
91475159102     1
91475159201     1
91475159300     1
91475407999     1
91475630045     1
91475630069     1
91475750002     1
Name: code, dtype: int64
________________________________________________________________________________________________________________________
unique non null values for : creator
creator
usda-ndb-import               169633
openfoodfacts-contributors     45774
kiliweb                        36374
date-limite-

### Peu de valeurs communes, ou des valeurs similaires entre colonnes
- allergens_en :contient des urls de visuels produits
- code : a l'air plutôt unique, que 3 produits partagent le code : 70650800367

In [ ]:
#On conserve pour la suite : product_name && code
#categories_en,main_category_en,pnns_groups_2,pnns_groups_1,nutrition_grade_fr,stores   
strings2Colonnes='generic_name,countries_tags,countries,ingredients_text,origins_tags,categories_tags,brands_tags,packaging_tags,states_tags,states_en,states,additives,serving_size,traces_en,traces_tags,traces,allergens_en,allergens,ingredients_text,cities_tags,cities,first_packaging_code_geo,emb_codes_tags,emb_codes,manufacturing_places_tags'.split(',')
#','.join(df.columns)
strings2=df[strings2Colonnes]#Lots Of NaN
allVars['strings2']=strings2
x='strings2.tsv';away.append(x);exportDf(strings2,x)
df=df.drop(strings2Colonnes,axis=1)  

put:http://1.x24.fr/a/jupyter/off/strings2.tsv.zip


### Liste des colonnes intéressantes
- Valeurs numériques
- Suffixe : _100g
- Puis listing des valeurs vides en % par colonne ( en incluant zéros et NaN )

In [ ]:
Cols100g=[]
#nbOccurencesMaximaleValeurParColonne={}
#Listing des valeurs les plus récurrentes par colonne
debug['emptyValues']={}
if True:  
  #i='ingredients_from_palm_oil_n'
  #df[df[i]==0].shape[0] == 277295
  #df[(df[i].isnull())].shape[0] == 72160
  #df[(df[i]==np.NaN)].shape[0] == 0
  #emptyValues=df[(df[i]==np.NaN) | (df[i]==0) | (df[i].isnull())].shape[0]#349455
  for i in df.columns:
    emptyValues=df[(df[i]==np.NaN) | (df[i]==0) | (df[i].isnull())].shape[0]#349455
    debug['emptyValues'][i]=round(emptyValues/rows,2)
    if('_100g' in i):
      Cols100g.append(i)

arsort(debug['emptyValues'])
allVars['Cols100g']=Cols100g
','.join(Cols100g)

'energy_100g,fat_100g,saturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,nutrition-score-fr_100g,nutrition-score-uk_100g'

## Recherche de Outliers ..

### Sur quoi se baser ? 
- Science / Savoir / Google
- BoxPlots / Violin plots : ces derniers sont un outil statistique, il se peuvent que certaines valeurs en dehors de la répartition de 90% des valeurs soient cohérentes .. notamment en présence ce valeurs nulles ou non renseignées !!

- Problématique : si une de ces colonne dispose d'une valeur abérrante le graphique va être déformé .. et ne plus montrer cette valeur, pour ceci on peut extraire les valeurs "déciles" : quantiles

=> Nb : il ne s'agit que de statistiques et de Probabilité calculée sur le jeu de données, ce qui serait très valable dans le cas de répétition de mesures physiques .. mais nous avons là de réels outliers ..

#### Colonnes non remplies
Combiens de colonnes ne sont pas remplies ???
- Puis on effectue des statistiques rapides selon toutes les colonnes
- On remplace tous les NaN par des 0 cela sera utile afin de réussir les corrélations

In [ ]:
df=cleanData(df,0);#pourrait être la valeur moyenne df[i].mean() ou via un modèle gaussianNB par exemple

Total Row*Columns in dataset : 16716866
Total null values : 15649832
____________________________________________________________________________________________________________________________________________
ingredients_from_palm_oil_tags                349109
ingredients_that_may_be_from_palm_oil_tags    341357
origins                                       330661
manufacturing_places                          313713
stores                                        298027
labels                                        296607
labels_tags                                   296527
labels_en                                     296527
purchase_places                               289169
packaging                                     265758
main_category                                 252472
main_category_en                              252472
categories_en                                 252446
categories                                    252446
quantity                                      23

#### Boxplots
- http://www.cabinetorientation.fr/img/qi-gauss-trans.png#courbeGaussOuBell
- http://1.x24.fr/a/jupyter/off/boxplots.webp

In [ ]:
#On ne converve que les colonnes numériques qui seront intéressantes à plotter
colonnesInt=list(df.select_dtypes(include=['float64','int64']).columns)
colonnesInt.remove('relevancy')

#https://www.inegalites.fr/IMG/jpg/deciles.jpg
#https://stackoverflow.com/questions/17725927/boxplots-in-matplotlib-markers-and-outliers

#variable à ajuster ..
seuilNonPertinenceAbsolu=1/10000;

j=0;axx=6;#par ligne
axy=math.ceil(len(colonnesInt)/axx);#Nb lignes
print(str(axx*axy)+' tot slots for '+str(len(colonnesInt))+' to display : ok')#=) provisionner plus d'emplacements n'est pas grave
f,axes=plt.subplots(axy,axx,figsize=(axy*heightPerGraph,axx*widthPerGraph))
f.patch.set_facecolor('white');axes=axes.flatten()#pour les parcourir plus facilement sur une seule dimension :)

for i in colonnesInt:
  x=df[(df[i]>0)][i]#on ignore les valeurs nulles ..
  Q00 = df[i].quantile(0+seuilNonPertinenceAbsolu)#might exclude lowest value
  Q100 = df[i].quantile(1-seuilNonPertinenceAbsolu)#the 17 max value, itself
  axes[j].boxplot(x[~np.isnan(x) & x[x<Q100] & x[x>Q00]]);#
  axes[j].set_title(i);
  j=j+1;#prochain ax
  continue;

f.savefig('boxplots.png', bbox_inches = 'tight')
webp('boxplots.png')
show()

if False:  
  #type(df[(df[i]>0)][i])#serie
  #type(df[(df[i]>0)])#dataframe
  #type(df[i])#Series
  #type(x)#dataframe
#np.warnings.filterwarnings('ignore')
#boxplot RuntimeWarning: invalid value encountered : NaN and Zeros  
#Mais des valeurs abérrantes rendent la lecture peu agréable si tout est ratatiné vers le bas
#retournés par df.describe#i='energy_100g'

  #<=Q99

  Q99 = df[i].quantile(0.99)
  Q25 = df[i].quantile(0.25)#le rectangle inclut dedu décile 25 à 75
  Q75 = df[i].quantile(0.75)#
  IQR = Q75 - Q25;#Range
  maxR=Q75+1.5*IQR
  maxL=Q25-1.5*IQR
  Q05 = df[i].quantile(0.05)
  Q95 = df[i].quantile(0.95)
  
  print([i,maxL,maxR])
  #on triche en filtrant les valeurs les plus abérrantes
  #sns.violinplot( y=df[j], x=df[i], ax=ax )
#prot max 71% fromage allégé

24 tot slots for 20 to display : ok
put:http://1.x24.fr/a/jupyter/off/boxplots.webp


' prot max 71% fromage allégé '

#### KDE
- Densité des répartions des données, ce qui permet de comprendre des absences passé une certaine valeur, peut servir également à déterminer les outliers .., hélàs elle nous affiche surtout les valeurs proches de 0, qui faussent l'axe vertical, et les valeurs abérrantes nous faussent l'axe horizontal .. peu exploitable avant un nettoyage des données
- http://1.x24.fr/a/jupyter/off/oldKdeNoLog.webp
- http://1.x24.fr/a/jupyter/off/kde2.webp

In [ ]:
sup0={}
for i in Cols100g:
  #exclude 0 values qui sont les plus fréquentes et ne laisseront pas apprécier les autres valeurs
  x=df[~df[i].isna() & df[i]>0][i]
  #ValueError: `dataset` input should have multiple elements.  caprylic
  if(len(x)>1):
    sup0[i]=x
    print(i+' has : '+str(len(x))+' records')
  
j=0;axx=5;axy=math.ceil(len(sup0)/axx);#tjrs 1 de plus
print(str(axx*axy)+' tot slots for '+str(len(Cols100g))+' to display : ok')#=) provisionner plus d'emplacements n'est pas grave
f,axes=plt.subplots(axy,axx,figsize=(axy*heightPerGraph,axx*widthPerGraph))
f.patch.set_facecolor('white');axes=axes.flatten()#pour les parcourir plus facilement sur une seule dimension :)

for i in sup0.keys():  
  sup0[i].plot(kind='kde',ax=axes[j],title=i).yscale='log';
  j=j+1;#prochain axe

f.savefig('oldKdeNoLog.png', bbox_inches = 'tight')
webp('oldKdeNoLog.png')  
show()

energy_100g has : 285951 records
fat_100g has : 211212 records
saturated-fat_100g has : 189786 records
trans-fat_100g has : 3070 records
cholesterol_100g has : 54676 records
carbohydrates_100g has : 255681 records
sugars_100g has : 239183 records
fiber_100g has : 146810 records
proteins_100g has : 237482 records
salt_100g has : 250988 records
sodium_100g has : 250984 records
vitamin-a_100g has : 59060 records
vitamin-c_100g has : 50535 records
calcium_100g has : 90454 records
iron_100g has : 97973 records
nutrition-score-fr_100g has : 240174 records
nutrition-score-uk_100g has : 238760 records
20 tot slots for 17 to display : ok
put:http://1.x24.fr/a/jupyter/off/oldKdeNoLog.webp


#### Analyse Valeurs ( opacity, log )
- En excluant les valeurs NAN & inférieures à 0 ===> Identification d'un outlier à l'aide d'un diagramme de dispersion
- Donc on compare en petits points avec faible opacité, puis l'inverse

In [ ]:
plt.rcParams["figure.figsize"] = (24,6)
i='energy_100g';
sup0=df[~df[i].isna() & df[i]>0][i];
scatter(sup0,i,'tinydots',fn='tinydots.png')
scatter(sup0,i,'bigdots-with-Outliers',size=20,opacity=1,fn='bigdots-with-Outliers.png',miny=sup0.quantile(0),maxy=sup0.quantile(1))#On visualise les outliers ICI
scatter(sup0,i,'bigdots-outliers-logs',yscale='log',size=20,opacity=1,fn='bigdots-outliers-logs.png',miny=sup0.quantile(0),maxy=sup0.quantile(1))#En mode Log Outliers en limite
scatter(sup0,i,'bigdots-quantiles',size=20,opacity=1,fn='bigdots-quantiles.png',miny=sup0.quantile(0.001),maxy=sup0.quantile(0.999))#excluant les outliers
#http://1.x24.fr/a/jupyter/off/scatter.tinydots.png.png  http://1.x24.fr/a/jupyter/off/scatter.bigdots.png.png 

put:http://1.x24.fr/a/jupyter/off/tinydots.webp
put:http://1.x24.fr/a/jupyter/off/bigdots-with-Outliers.webp
put:http://1.x24.fr/a/jupyter/off/bigdots-outliers-logs.webp
put:http://1.x24.fr/a/jupyter/off/bigdots-quantiles.webp


1

##### => Echelle Log
=> Il faut également fixer un cap ou une échelle logarythmique, sinon les "outliers" ratatinent certains graph et l'on ne voit rien .. http://1.x24.fr/a/jupyter/off/scatterHeatDensityCompared.png

In [ ]:
#resume()
plt.rcParams["figure.figsize"] = (26,6)
cols=list(df.columns.values)
cols.remove('relevancy')

sup0={}
for i in cols:
  t=str(df[i].dtypes)
  #no numeric data to plot and (type(df[i][0])!=int
  if((t=='float64') or (t=='int64')):
    #Filtrer les NaN et Valeurs = 0 => on gagne de l'espace sur l'axe horizontal ( cela est mieux pour distinguer les valeurs rares )
    x=df[~df[i].isna() & df[i]>0][i]
    if(len(x)>1):
      sup0[i]=x

Nb=2
tot=len(sup0)*Nb
j=0;axx=Nb;axy=math.ceil(tot/axx);#vertical limit
print('Columns:'+str(axx)+',Rows:'+str(axy)+'=>'+str(axx*axy)+' tot slots for '+str(tot)+' graphs to display : ok')#=) provisionner plus d'emplacements n'est pas grave
f,axes=plt.subplots(axy,axx,figsize=(axx*widthPerGraph,axy*heightPerGraph))
f.patch.set_facecolor('white');axes=axes.flatten()#pour les parcourir plus facilement sur une seule dimension :)

plt.ioff()#matplotlib output off > lighter notebook exported != show() or %matplotlib inline
heightPerGraph=widthPerGraph=6
plt.rcParams["figure.figsize"] = (heightPerGraph,widthPerGraph)

for i in sup0.keys():    
  print(i+':'+str(sup0[i].size))#97869
  #sup0[i].plot(kind='kde',ax=axes[j],title=i);
  miny=sup0[i].quantile(lowQuantileLimit);
  maxy=limitFor(i)
  scatter(sup0[i],i,'log-low',yscale='log',axis=axes[j],maxy=maxy);j=j+1;#color=orange
  if(sup0[i].quantile(highQuantileLimit)<maxy):
    maxy=sup0[i].quantile(highQuantileLimit);#ré-adapter la hauteur du graph
  scatter(sup0[i],i,'quantileLimit',size=10,opacity=0.5,axis=axes[j],miny=miny,maxy=maxy);j=j+1;#iron_100g => what about the few points,maxy=sup0[i].quantile(0.999)
  #heat(sup0[i],i,'heat',yscale='log',axis=axes[j],s=1,bins=300,vmax=300);j=j+1;
  #prochain axe

x='scatterPlotsOpacityLimitsLogCompared.png'
f.savefig(x, bbox_inches = 'tight')
webp(x)  
show()

Columns:2,Rows:22=>44 tot slots for 44 graphs to display : ok
sum:292363
kj:285879
additives_n:170189
ingredients_from_palm_oil_n:6572
ingredients_that_may_be_from_palm_oil_n:14328
energy_100g:285951
fat_100g:211212
saturated-fat_100g:175591
trans-fat_100g:3035
cholesterol_100g:54676
carbohydrates_100g:255681
sugars_100g:223922
fiber_100g:146809
proteins_100g:237479
salt_100g:250988
sodium_100g:250984
vitamin-a_100g:59059
vitamin-c_100g:50534
calcium_100g:90454
iron_100g:97972
nutrition-score-fr_100g:240174
nutrition-score-uk_100g:238760
put:http://1.x24.fr/a/jupyter/off/scatterPlotsOpacityLimitsLogCompared.webp


#### Constats 
- pour le cholesterol nous avons 54598 records, la plupart très proches de 0 donc les points n'apparaissent pas sur le graphe

In [ ]:
i='cholesterol_100g';
df[~df[i].isna() & df[i]>0][i];#54598 records valides, très proche de 0

1         0.018
67        0.077
72        0.009
109       0.012
150       0.018
          ...  
355821    0.071
355859    0.009
355860    0.009
355884    0.007
355899    0.107
Name: cholesterol_100g, Length: 54598, dtype: float64

### Les faits établis
- Le nutrionscore peut être négatif, les autres valeurs pour la plupart doivent être comprises en 0 et 100g ( proportion dans leurs aliments respectifs )

- Un aliment 100% de lipides : c'est de l'huile ( > 90% lipides ) devient liquide à température ambiante > 22°, l'huile n'a pas la même masse que l'eau ( densité huile:0.92 1kg eau = 1Litre )
, elle est plus légère 1 Litre d'huile vaut 3766KJ et pèse 920grammes c'est donc le maximum que l'on peut trouver dans un aliment, soit 4093KJoules par kilo qui deviendra donc l'unité de rejet pour energy_100g

- On prend toutes les colonnes aux valeurs numérique, on rajoute la valeur médiane, et l'on identifie visuellement les "outliers"

- Il ne peut y avoir plus de sucre que de glucides au total .. cela vaut pour toutes les relations parent/enfant

D'après https://en.wiki.openfoodfacts.org/API/Read/Product nutrion score peut être négatif .. notamment pour l'eau
- cocoa, fruits, nuts : en pourcentage, ph en ph

Ainsi, comparer les valeurs via describe ( entre 75% et max ) permet de se faire une bonne idée de ces valeurs abérrantes à corriger

- Huile : somme des valeurs : 100% de gras, mais certains aliments contiennent de l'eau ( viande moyenne 70% ) ou excipients ou gélifiants, édulcorants etc..

In [ ]:
df.describe()

,relevancy,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,energy_100g,fat_100g,saturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
count,355678.0,355678.000000,355678.000000,355678.000000,355678.000000,3.556780e+05,355678.000000,355678.000000,355678.000000,3.556780e+05,355678.000000,3.556780e+05,3.556780e+05,355678.000000,355678.000000,355678.000000,355678.000000,355678.000000,355678.000000,355678.000000,355678.000000
mean,100.0,1.496494,0.018691,0.047650,933.702751,4.405737e+04,3.771834,0.029632,0.008118,4.407205e+04,12.293733,2.384707e+05,4.405325e+04,1.579936,0.622020,0.000154,0.009276,0.050520,0.001499,6.560161,6.427392
std,0.0,2.357521,0.137062,0.251808,952.519262,2.626927e+07,7.207881,0.978275,0.228111,2.626927e+07,20.511884,1.422199e+08,2.626927e+07,108.802259,42.835534,0.045572,1.407507,2.113518,0.136203,8.664002,8.738143
min,100.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-3.570000,0.000000,0.000000e+00,-17.860000,-6.700000e+00,-8.000000e+02,0.000000,0.000000,-0.000340,-0.002100,0.000000,-0.000260,-15.000000,-15.000000
25%,100.0,0.000000,0.000000,0.000000,146.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,100.0,0.000000,0.000000,0.000000,787.000000,1.700000e+00,0.200000,0.000000,0.000000,1.083000e+01,3.000000,0.000000e+00,3.200000e+00,0.200000,0.078740,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000
75%,100.0,2.000000,0.000000,0.000000,1569.000000,1.500000e+01,4.710000,0.000000,0.000000,5.000000e+01,15.000000,2.200000e+00,8.510000e+00,1.122680,0.442000,0.000000,0.000000,0.008000,0.000430,14.000000,14.000000
max,100.0,30.000000,2.000000,6.000000,231199.000000,1.566667e+10,550.000000,369.000000,95.238000,1.566667e+10,3520.000000,8.481815e+10,1.566667e+10,64312.800000,25320.000000,26.700000,716.981100,694.737000,50.000000,40.000000,40.000000


## Nettoyage

### Valeurs maximales possibles

In [ ]:
def limitFor(i):
  limit=100
  if('carbon' in i):
    return 999999999;#footprint
  if('fiber' in i):
    return 52;#canelle
  if('energy' in i):    
    return 3700;#huile 100g considéré comme 100ml, les gens ne font pas vraiment la différence : 4093  
  if('cholesterol' in i):
    return 3.1;#cervelle de veau : 3g max pour 100mg aliment
  if('calcium' in i):
    return 2;#	meloukhia en poudre ??
  if('iron' in i):
    return 0.2;
#A:beta catorène:30mg foie de morue, 200mg C pour Cassis    
  if('vitamin-a' in i):
    return 0.03;
  if('vitamin-c' in i):
    return 0.2;
  if('sodium' in i):
    return 40;
  if('nutrition-score' in i):
    return 40;
  return limit

#exportDf(df,'temp.tsv');#df=loadData('temp.tsv')
if True:# or 'disqualifiate bad data' MARK II
  for k in Cols100g:
    limit=limitFor(k)
#ph entre 1 et 14
    x=df.loc[(df[k]<0)].shape[0];
    #allow negative nutriontion score
    #unsupported operand type(s) for &: 'int' and 'str'
    #k='nutrition-score-fr_100g'
    if((k!='nutrition-score-fr_100g') & (k!='nutrition-score-uk_100g') & ('nutrition' not in k) & x>0):    
      #print(k)
      outliers[k+'<0']=x
      print('Valeurs <0:'+k)
      print(df.loc[(df[k]<0)][k][:5])
      df.loc[(df[k]<0),k]=0;#no negative values !
      df.loc[(df[k]<0),'relevancy']=df['relevancy']-30;
      df.loc[(df[k]<0),'notes']=df['notes']+','+k+'<0';

    x=df.loc[(df[k]>limit)].shape[0];
    if(x>0):
      outliers[k+'>'+str(limit)]=x
      print('Valeurs trop elevées : '+k)
      print(df.loc[(df[k]>limit)][k][:5])
      print('_'*120)
      df.loc[(df[k]>limit),k]=limit;#affecter à leur limites respective en baissant leur pertinence !
      df.loc[(df[k]>limit),'relevancy']=df['relevancy']-30;
      df.loc[(df[k]>limit),'notes']=df['notes']+','+k+'>'+str(limit);

arsort(outliers)      

Valeurs trop elevées : energy_100g
736     3887.0
812     4435.0
3838    3837.0
3862    3837.0
5494    5690.0
Name: energy_100g, dtype: float64
________________________________________________________________________________________________________________________
Valeurs trop elevées : fat_100g
177004    1.566667e+10
178692    2.000000e+02
334384    1.010000e+02
336897    1.050000e+02
Name: fat_100g, dtype: float64
________________________________________________________________________________________________________________________
Valeurs trop elevées : saturated-fat_100g
120771    175.38
347260    550.00
Name: saturated-fat_100g, dtype: float64
________________________________________________________________________________________________________________________
Valeurs trop elevées : trans-fat_100g
12036     369.00
82331     130.00
82333     130.00
122346    110.00
140017    173.26
Name: trans-fat_100g, dtype: float64
_____________________________________________________________

[('vitamin-c_100g>0.2', 929),
 ('energy_100g>3700', 791),
 ('salt_100g>100', 164),
 ('fiber_100g>52', 148),
 ('calcium_100g>2', 136),
 ('sodium_100g>40', 118),
 ('iron_100g>0.2', 35),
 ('carbohydrates_100g>100', 21),
 ('vitamin-a_100g>0.03', 16),
 ('sugars_100g>100', 13),
 ('sugars_100g<0', 7),
 ('trans-fat_100g>100', 5),
 ('cholesterol_100g>3.1', 5),
 ('proteins_100g>100', 5),
 ('fat_100g>100', 4),
 ('proteins_100g<0', 3),
 ('saturated-fat_100g>100', 2),
 ('fiber_100g<0', 1),
 ('vitamin-a_100g<0', 1),
 ('vitamin-c_100g<0', 1),
 ('iron_100g<0', 1)]

#### Somme
- On crée une colonne "sum" ( sommes des grammages présents .. ), selon ses valeurs respectives on retranche de la pertinences aux lignes
- On n'additionne pas les relations enfant ( sodium € sel, sucres #carbohydrates etc .. ) sinon la somme sera toujours > 100

> - NB : La somme des ingrédients constitutifs peut être comprise entre 0 et 100 : rempli soit par de l'eau, de l'alcool ou des agents aux valeurs nutrionitielles inertes ( ex : cendres dans les croquettes pour chats )


In [ ]:
if True:
  if('sum' not in df.columns.values):
    df.insert(2,'sum',0)
    
  sumCols=Cols100g.copy()
#sumCols='fat_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-montanic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-gondoic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g'.split(',')  
  enfantsANePasAdditioner='energy_100g,trans-fat_100g,saturated-fat_100g,sodium_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g'.split(',')
  OffSpring={'carbohydrates_100g':'sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g'.split(','),'fat_100g':'trans-fat_100g,saturated-fat_100g'.split(','),'salt_100g':['sodium_100g']}
#sugars_100g>carbohydrates_100g : 693
#Elimination par relations parent / enfants : un enfant ne peut avoir plus de valeur que son parent
  for i in OffSpring.keys():
    for j in OffSpring[i]:
      if(j not in df.columns):
        continue;#column deleted above
      x=df.loc[(df[j]>df[i])]
      if(x.shape[0]>0):
        outliers[j+'>'+i]=x.shape[0]
        for k in x.index.tolist():
          df.at[k,j]=df.at[k,i]#set the parent value
          df.at[k,'relevancy']=df.at[k,'relevancy']-20
          df.at[k,'notes']=df.at[k,'notes']+','+j+'>'+i

  for i in enfantsANePasAdditioner:
    if(i in sumCols):
      print('Ne pas additionner ces colonnes:'+i)
      sumCols.remove(i)

  df['sum']=df[sumCols].sum(axis=1,numeric_only=True,skipna=True)

  x=df.loc[(df['sum']>100)]
  outliers['sum>100']=x.shape[0]  
  if(x.shape[0]>0):
    for i in x.index.tolist():
      df.at[i,'relevancy']=df.at[i,'relevancy']-20
      df.at[i,'notes']=df.at[i,'notes']+',sum>100'

  x=df.loc[(df['sum']<1)];
  outliers['sum<1']=x.shape[0]
  if(x.shape[0]>0):
    for i in x.index.tolist():
      df.at[i,'relevancy']=df.at[i,'relevancy']-100
      df.at[i,'notes']=df.at[i,'notes']+',sum<1'
#La somme des ingrédients constitutifs peut être comprise entre 0 et 100 : rempli soit par de l'eau, de l'alcool ou des agents aux valeurs nutrionitielles inertes
  if False:
    x=df.loc[(df['sum']<=70)&(df['sum']>60)];#.astype('int')
    outliers['sum<=70']=x.shape[0]     
    for i in x.index.tolist():
      df.at[i,'relevancy']=df.at[i,'relevancy']-40
      df.at[i,'notes']=df.at[i,'notes']+',sum<70'

    x=df.loc[(df['sum']<=60)&(df['sum']>50)];
    outliers['outliers:sum<=60']=x.shape[0]   
    for i in x.index.tolist():
      df.at[i,'relevancy']=df.at[i,'relevancy']-50
      df.at[i,'notes']=df.at[i,'notes']+',sum<60'

    x=df.loc[(df['sum']<=50)&(df['sum']>40)];
    outliers['outliers:sum<=50']=x.shape[0]
    for i in x.index.tolist():
      df.at[i,'relevancy']=df.at[i,'relevancy']-60
      df.at[i,'notes']=df.at[i,'notes']+',sum<50'

    x=df.loc[(df['sum']<=40)&(df['sum']>30)]
    outliers['outliers:sum<=40']=x.shape[0]
    for i in x.index.tolist():
      df.at[i,'relevancy']=df.at[i,'relevancy']-70
      df.at[i,'notes']=df.at[i,'notes']+',sum<40'
    
    x=df.loc[(df['sum']<=30)&(df['sum']>20)]
    outliers['outliers:sum<=30']=x.shape[0]
    for i in x.index.tolist():
      df.at[i,'relevancy']=df.at[i,'relevancy']-80
      df.at[i,'notes']=df.at[i,'notes']+',sum<30'

    x=df.loc[(df['sum']<=20)&(df['sum']>1)]
    if(x.shape[0]>0):
      outliers['outliers:sum<=20']=x.shape[0]
      for i in x.index.tolist():
        df.at[i,'relevancy']=df.at[i,'relevancy']-90
        df.at[i,'notes']=df.at[i,'notes']+',sum<20'
  
#df[[sumCols]].map(sum);columns=list(df)
#some are integers, others are dict ..
print(dfQuery(df,'code','3065620011192'))#or index:197147
arsort(outliers)

Ne pas additionner ces colonnes:energy_100g
Ne pas additionner ces colonnes:trans-fat_100g
Ne pas additionner ces colonnes:saturated-fat_100g
Ne pas additionner ces colonnes:sodium_100g
Ne pas additionner ces colonnes:sugars_100g
        relevancy notes    sum           code  creator product_name quantity packaging      brands categories categories_en origins manufacturing_places labels labels_tags labels_en purchase_places stores countries_en  additives_n  ... pnns_groups_2 main_category  main_category_en energy_100g  fat_100g saturated-fat_100g trans-fat_100g cholesterol_100g carbohydrates_100g sugars_100g fiber_100g  proteins_100g  salt_100g  sodium_100g  vitamin-a_100g  vitamin-c_100g  calcium_100g  iron_100g  nutrition-score-fr_100g  nutrition-score-uk_100g
197165        100        79.12  3065620011192  kiliweb     Spirales        0         0  Grand Mère          0             0       0                    0      0           0         0               0      0       France          

[('sum>100', 90686),
 ('sum<1', 69214),
 ('sugars_100g>carbohydrates_100g', 15743),
 ('saturated-fat_100g>fat_100g', 14424),
 ('vitamin-c_100g>0.2', 929),
 ('energy_100g>3700', 791),
 ('salt_100g>100', 164),
 ('fiber_100g>52', 148),
 ('calcium_100g>2', 136),
 ('sodium_100g>40', 118),
 ('trans-fat_100g>fat_100g', 62),
 ('iron_100g>0.2', 35),
 ('carbohydrates_100g>100', 21),
 ('vitamin-a_100g>0.03', 16),
 ('sugars_100g>100', 13),
 ('sugars_100g<0', 7),
 ('trans-fat_100g>100', 5),
 ('cholesterol_100g>3.1', 5),
 ('proteins_100g>100', 5),
 ('fat_100g>100', 4),
 ('proteins_100g<0', 3),
 ('saturated-fat_100g>100', 2),
 ('fiber_100g<0', 1),
 ('vitamin-a_100g<0', 1),
 ('vitamin-c_100g<0', 1),
 ('iron_100g<0', 1)]

#### KJ
- Carbohydrates = classification générales pour les glucides, il est normal qu'il y en ait d'autant plus que de sucre ..
- Reconstruisons un index en kj à partir des données génériques de la base de données initiale ..
- Inspectons les écart les plus gros .. certains aliments ont une somme entre le gras, les protéines et les glucides de 300g pour 100g .. ces anomalies sont déjà reportées sur le score de pertinence ..


In [ ]:
if('kj' not in df.columns.values):
  df.insert(3,'kj',0)
    
df.fat_100g * 37 + (df.proteins_100g + df.carbohydrates_100g)* 17
df["kj"] = df.fat_100g * 37 + (df.proteins_100g + df.carbohydrates_100g)* 17
df["kj"]
#1700+1700+3700 ?
if False:#Exploration manuelle
  #écart entre énergie déclarée et énergie déduite des colonnes
  abs(df["kj"]-df['energy_100g']).sort_values(ascending=True)[:40];#les moins abérrantes car pertinentes
  abs(df["kj"]-df['energy_100g']).sort_values(ascending=False)[:40];#7100 Kj n'est ce pas un peu beaucoup ? 

  #Inspection manuelle des anomalies rencontrées
  df.loc[71633,:]['revelancy,product_name,notes,sum,kj,energy_100g,proteins_100g,fat_100g,carbohydrates_100g'.split(',')]#quand rien n'est renseigné .. il ne risque pas d'y avoir d'écarts ..
  df.loc[177004,:]['revelancy,product_name,notes,sum,kj,energy_100g,proteins_100g,fat_100g,carbohydrates_100g'.split(',')]#get Row per index Name, has a sum of 300
  df.loc[178692,:]['revelancy,product_name,notes,sum,kj,energy_100g,proteins_100g,fat_100g,carbohydrates_100g'.split(',')]
  #df["energy_100g"].sort_values(ascending=False)#4093
  #df["carbohydrates_100g"].sort_values(ascending=False)

  df["kj"].sort_values(ascending=False).index
  df.loc[1,:]['revelancy,notes,sum,kj,energy_100g,proteins_100g,fat_100g,carbohydrates_100g'.split(',')]#get Row per index Name, has a su of 300

### KDE2
- On constate que ce graphique est beaucoup plus simple à interprêter une fois les données nettoyées de leurs valeurs abérrantes 
- http://1.x24.fr/a/jupyter/off/kde2.webp

In [ ]:
sup0={}
for i in Cols100g:
  #exclude 0 values qui sont les plus fréquentes et ne laisseront pas apprécier les autres valeurs
  x=df[~df[i].isna() & df[i]>0][i]
  #ValueError: `dataset` input should have multiple elements.  caprylic
  if(len(x)>1):
    sup0[i]=x
    print(i+' has : '+str(len(x))+' records')
  
j=0;axx=5;axy=math.ceil(len(sup0)/axx);#tjrs 1 de plus
print(str(axx*axy)+' tot slots for '+str(len(Cols100g))+' to display : ok')#=) provisionner plus d'emplacements n'est pas grave
f,axes=plt.subplots(axy,axx,figsize=(axy*heightPerGraph,axx*widthPerGraph))
f.patch.set_facecolor('white');axes=axes.flatten()#pour les parcourir plus facilement sur une seule dimension :)

for i in sup0.keys():  
  ax=axes[j]
  mini=0;
  if('nutrition' in i):
    mini=-10;
  ax.set_xlim(mini,limitFor(i))#Avec les limites adaptées à chaque type de données
  sup0[i].plot(kind='kde',ax=ax,title=i);
  j=j+1;#prochain axe

x='kde2.png'
f.savefig(x, bbox_inches='tight')
webp(x)  
show()

energy_100g has : 285951 records
fat_100g has : 211212 records
saturated-fat_100g has : 175591 records
trans-fat_100g has : 3035 records
cholesterol_100g has : 54676 records
carbohydrates_100g has : 255681 records
sugars_100g has : 223922 records
fiber_100g has : 146809 records
proteins_100g has : 237479 records
salt_100g has : 250988 records
sodium_100g has : 250984 records
vitamin-a_100g has : 59059 records
vitamin-c_100g has : 50534 records
calcium_100g has : 90454 records
iron_100g has : 97972 records
nutrition-score-fr_100g has : 240174 records
nutrition-score-uk_100g has : 238760 records
20 tot slots for 17 to display : ok
put:http://1.x24.fr/a/jupyter/off/kde2.webp


#### => Exportation du dataframe nettoyé à utiliser sur application

In [ ]:
#if(getFile(fn2)):
#  df=pd.read_csv(fn2, sep='\t')
exportDf(df,fn2)#Cleaned Dataframe location
#ftpput(fn2)

allVars['df']=df
allVars['outliers']=outliers
allVars['debug']=debug

FPC('allVars',allVars)#allVars.keys()
ftpput('allVars.pickle')#resume()
say('cleaned dataset exported')
print(allVars.keys())

put:http://1.x24.fr/a/jupyter/off/en.openfoodfacts.org.products.tsv2.tsv.zip
put:http://1.x24.fr/a/jupyter/off/allVars.pickle.zip
put:http://1.x24.fr/a/jupyter/off/allVars.pickle.zip


dict_keys(['valeursExotiques', 'strings1', 'strings2', 'Cols100g', 'df', 'outliers', 'debug'])


## Matrice de corrélation primitive ..
- Afin de prendre de la hauteur et avoir une vision d'ensemble des données, il convient de réaliser une grande matrice de corrélation permettant d'inspecter par la suite les relations potentielles entre deux variables en prennant bien soin de ne prendre que les valeurs numériques .. sinon les scores de 1 font état de doublons entre colonnes non préalablement nettoyées
- Trois méthodes : pearson, kendall, & spearman ..
- NB : il peut exister des corrélations qui n'ait aucun lien de cause à effet ex :  https://s3.eu-west-1.amazonaws.com/user.oc-static.com/upload/2017/11/20/1511218349727_ex_SpuriousCorrelation.jpeg
<hr>

> On trie ces dernières par corrélations les plus remarquables ! En excluant bien entendu les valeurs à 0 qui auront tendance à corréler d'autant plus qu'elles sont nombreuses !

In [ ]:
#Global HeatMap .. so much rows > dumped to png file for reviewing
dropped=[]
factorized=[]
heatmapCols=Cols100g.copy()
"""
df2=df.dropna()
df2 = df.fillna(df.mean())
df = df.apply(pd.to_numeric, errors='coerce')
"""

def corr2(a,b):
  v = np.minimum(a, b).sum().round(decimals=1)
  return v

#heatmapCols=heatmapCols+['sum','kj']#ajout des deux colonnes "computées"
if True:
#si le dataframe n'est pas défini ( en cas de plantage notamment )  
  if('df' not in locals().keys()):
    df=loadData(fn)
  heatmap=df[heatmapCols].copy(deep=True)#ne jamais écraser le dataframe de travail ..

#msk = (~a.mask & ~b.mask)
#heatmap.corrcoef(np.ma.masked_invalid(df['energy_100g']), np.ma.masked_invalid(df['fat_100g']))

  cols=heatmap.columns
  for i in cols:#heatmap.dtypes[i][0] same as type(heatmap['manufacturing_places_tags'].values.dtype)
    t=str(heatmap[i].values.dtype)
    #print('>>'+i+':'+t)#dtype:object
    #drop each string columns not being relevant here
    if((t!='float64') and (t!='int64')):
      heatmap=heatmap.drop([i],axis=1)#key error : allergens, whut ?
      dropped.append(i)  
  for i in heatmap.columns.values:
    if 'numpy' not in str(type(heatmap[i][0])): 
      heatmap[i],levels=pd.factorize(heatmap[i])#exclure les valeurs passées à 0
      factorized.append(i) 
  print('Notice : Factorized string columns :')
  print(','.join(factorized))
  print('Notice : dropped non numeric columns :')
  print(','.join(dropped))

  for i in heatmap.columns.values:
    if 'numpy' not in str(type(heatmap[i][0])): 
      heatmap[i],levels=pd.factorize(heatmap[i])

#exclure des corrélations si colonne non remplie à plus de 10%
  minp=round(heatmap.shape[0]*0.1)
#le sel
#custom=heatmap.corr(method=corr2,min_periods=minp)

  correlationMethods={}
  methods='pearson,kendall,spearman'.split(',')
  print('Méthodes et Top corrélations: TOP 10')
  for i in methods:
    correlationMethods[i]=heatmap.corr(method=i,min_periods=minp)
    print('_'*140);
    print('>> '+i)
    print(correlationMethods[i].where(np.triu(np.ones(correlationMethods[i].shape),k=1).astype(np.bool)).stack().sort_values(ascending=False)[:10])

Notice : Factorized string columns :

Notice : dropped non numeric columns :

Méthodes et Top corrélations: TOP 10
____________________________________________________________________________________________________________________________________________
>> pearson
salt_100g                sodium_100g                0.999986
nutrition-score-fr_100g  nutrition-score-uk_100g    0.983245
energy_100g              fat_100g                   0.753060
fat_100g                 saturated-fat_100g         0.722170
carbohydrates_100g       sugars_100g                0.705105
energy_100g              nutrition-score-uk_100g    0.659106
                         nutrition-score-fr_100g    0.635370
saturated-fat_100g       nutrition-score-uk_100g    0.612868
                         nutrition-score-fr_100g    0.592830
energy_100g              saturated-fat_100g         0.575899
dtype: float64
____________________________________________________________________________________________________________

##### Comparaison

In [ ]:
plt.rcParams["figure.figsize"]=(heatmap.shape[1],heatmap.shape[1])#Massive Output to register to jpeg as much columns * 45px

for i in methods:
  np.fill_diagonal(correlationMethods[i].values, np.nan)#no more ones diagonal

  ax=sns.heatmap(correlationMethods[i],xticklabels=correlationMethods[i].columns.values,yticklabels=correlationMethods[i].columns.values,annot=True,cmap="YlGnBu")
  ax.xaxis.set_ticks_position('top')
  for j in ax.get_xticklabels():
    j.set_rotation(90)

  x='correlations-'+i+'.png'
  ax.get_figure().savefig(x, bbox_inches = 'tight');
  webp(x)
  show()

put:http://1.x24.fr/a/jupyter/off/correlations-pearson.webp
put:http://1.x24.fr/a/jupyter/off/correlations-kendall.webp
put:http://1.x24.fr/a/jupyter/off/correlations-spearman.webp


###### Quelles corrélations retenir ?
- Pearson est la méthode par défaut, chacune a ses avantages et inconvénients ..

https://towardsdatascience.com/pearson-coefficient-of-correlation-explained-369991d93404?# Pearson => linéaire très sensible aux outliers
https://towardsdatascience.com/kendall-rank-correlation-explained-dee01d99c535?# Kendall Tau-B => Peu d'échantillons, classés en quantités ou amounts ( _100g )
https://towardsdatascience.com/getting-the-basics-of-correlation-covariance-c8fc110b90b4# Spearman Rho => linéaire, tolère des variations


In [ ]:
correlations=correlationMethods['pearson']
np.fill_diagonal(correlations.values, np.nan)#no more ones diagonal
FPC('off-AllCorrelations',correlations)
correlations

#### Meilleures corrélations
- On constate : il y a du sodium dans le sel ( 39.5g pour 100g de sel ) , corrélation de 99%
- Le calcul du nutritionscore_uk est très similaire au nutrionscore_fr pour 96 %
- Bonne corrélation entre l'énergie calculée (kj) contre energy_100g ( 92% )

- Les graisses saturée ont une forte affinité avec le gras ( relation parent/enfant ) 71%
- L'apport calorique total d'un produit corrèle à 58% avec sa teneur en gras ( 61%)
- Les sucres sont des glucides ( 57% )
- Le score de nutrion a une forte corrélation avec la teneur en graisse, apport calorique d'un aliment
- Le cholestérol a une affinité avec les graisses saturées 39%
- Ka présence de vitamine C est affine avec la vitamine A à 30%



In [ ]:
correlations['cholesterol_100g'].sort_values(ascending=False)[:10]  
correlations.where(np.triu(np.ones(correlations.shape),k=1).astype(np.bool)).stack().sort_values(ascending=False)[:30]

salt_100g                sodium_100g                0.999986
nutrition-score-fr_100g  nutrition-score-uk_100g    0.983237
energy_100g              fat_100g                   0.753017
fat_100g                 saturated-fat_100g         0.722061
carbohydrates_100g       sugars_100g                0.705140
energy_100g              nutrition-score-uk_100g    0.659004
                         nutrition-score-fr_100g    0.635256
saturated-fat_100g       nutrition-score-uk_100g    0.612832
                         nutrition-score-fr_100g    0.592775
energy_100g              saturated-fat_100g         0.575891
                         carbohydrates_100g         0.562381
fat_100g                 nutrition-score-uk_100g    0.530421
                         nutrition-score-fr_100g    0.501752
sugars_100g              nutrition-score-fr_100g    0.420080
                         nutrition-score-uk_100g    0.411071
energy_100g              proteins_100g              0.393909
                        

##### Faux positifs !
- Kendall et Spearman retournent une corrélation entre cholestérol, vitamine A et Calcium ..
- Kendall est sensible aux faibles distributions et ces indices sont toujours sur des valeurs faibles ..
- Cette corrélation constitue une droite vertical pour les valeurs en x à 0 .. donc de faux positifs le cas échéant
- => Un graph de type heatmap est bien plus utile à comprendre leurs répartitions

In [ ]:
plt.rcParams["figure.figsize"] = (heightPerGraph,widthPerGraph)
#Les valeurs en cholestérol semblent réparties là où la vitamine A n'est pas renseignée ..
scatter(df,'vitamin-a_100g','cholesterol_100g',size=20,opacity=1)#48 spear
#Même constat pour le calcium
scatter(df,'calcium_100g','cholesterol_100g',size=20,opacity=1)#50 spear
#Pearson nous retourne 33 sur cette relation et cela semble plus pertinent au premier abord ..
scatter(df,'proteins_100g','cholesterol_100g',size=20,opacity=1)        
#Moins dense, et bien plus précise !
scatter(df,'proteins_100g','cholesterol_100g') 

#Heatmap permet une meilleur interprétation, plus rapide .. avec un lissage gaussien minimum ( groupe les points par proximité )
heat(df,'proteins_100g','cholesterol_100g',bins=100,vmax=50)

put:http://1.x24.fr/a/jupyter/off/vitamin-a_100g.cholesterol_100g.webp
put:http://1.x24.fr/a/jupyter/off/calcium_100g.cholesterol_100g.webp
put:http://1.x24.fr/a/jupyter/off/proteins_100g.cholesterol_100g.webp
put:http://1.x24.fr/a/jupyter/off/heat.proteins_100g.cholesterol_100g.png
put:http://1.x24.fr/a/jupyter/off/proteins_100g.cholesterol_100g.webp


1

In [ ]:
#Meilleures corrélations par index et Valeurs Absolues !
for i in correlations:
  print('_'*140)
  print(i+'>>>')
  print(correlations[i].abs().sort_values(ascending=False)[:5])

____________________________________________________________________________________________________________________________________________
energy_100g>>>
fat_100g                   0.753017
nutrition-score-uk_100g    0.659004
nutrition-score-fr_100g    0.635256
saturated-fat_100g         0.575891
carbohydrates_100g         0.562381
Name: energy_100g, dtype: float64
____________________________________________________________________________________________________________________________________________
fat_100g>>>
energy_100g                0.753017
saturated-fat_100g         0.722061
nutrition-score-uk_100g    0.530421
nutrition-score-fr_100g    0.501752
proteins_100g              0.277328
Name: fat_100g, dtype: float64
____________________________________________________________________________________________________________________________________________
saturated-fat_100g>>>
fat_100g                   0.722061
nutrition-score-uk_100g    0.612832
nutrition-score-fr_100g    0.59

Corrélations à 100 %
> Après avoir enlevé des colonnes on retrouve des corrélations à 100% qui ne sont pas logique, nous allons donc filter les indicateurs calories/gras/protéines/sucres/sel/sodium dont les valeurs sont inférieures ou égales à 0

### Allure des corrélations (Pairplots)
- Permet d'établir les véritables liens
- http://1.x24.fr/a/jupyter/off/?q=pairplots-

In [ ]:
heightPerUnit=69336/107/9;#72
le=len(heatmap.columns)-2#2 removed rows
#ne jamais excéder 65536 pixels par dimension ..
maxh=65536/72/le;
graphHeight=8
if(graphHeight>maxh):
  graphHeight=int(maxh)

mCols=list(heatmap.columns.values)
#mCols.remove('sum');mCols.remove('kj')

for i in mCols:  
  inc=0
  #i='energy_100g'
  cols=mCols.copy()
  cols.remove(i);#remove self
#ValueError: Image size of 648x69336 pixels is too large. It must be less than 2^16(‭65536‬) in each direction.  
#Limit Y axis for pairplots ! #reg takes a WHILE !!!!
#DataFrame.corrwith(self, other, axis=0, drop=False, method='pearson')
  maxi=limitFor(i);
  if(df[i].max()<maxi):
    maxi=df[i].max()

  g=sns.pairplot(heatmap,kind='scatter',height=graphHeight,aspect=1,x_vars=cols,y_vars=[i],plot_kws={'alpha':0.05},markers="+");#width=height*aspect,hue='LargestPropertyUseType'#,y_vars
  g.set(ylim=(0,maxi))#main
  
  for c in cols:
    maxi=limitFor(c);
    if(df[c].max()<maxi):
      maxi=df[c].max()
    g.axes[0,inc].set_xlim(0,maxi)#-2 pour voir les dots
    inc=inc+1;
  x='pairplots-'+i+'.png';
  g.fig.savefig(x, bbox_inches = 'tight');
  webp(x);
  show()

put:http://1.x24.fr/a/jupyter/off/pairplots-energy_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-fat_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-saturated-fat_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-trans-fat_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-cholesterol_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-carbohydrates_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-sugars_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-fiber_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-proteins_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-salt_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-sodium_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-vitamin-a_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-vitamin-c_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-calcium_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-iron_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplots-nutrition-score-fr

#### => Attention !
- la taille des points ainsi que leur opacité peut tromper sur la réelle répartition de données

In [ ]:
plt.rcParams["figure.figsize"] = (6,24)
#size=20 et opacity=1 sont les paramètres par défaut du scatter plot
scatter(df,'fat_100g','energy_100g',maxx=100,maxy=4000,size=20,opacity=1,fn='fat-energy-bigDots.png')
scatter(df,'fat_100g','energy_100g',maxx=100,maxy=4000,size=2,fn='fat-energy-tinyDots.png')#en les ajustant nous pouvons déjà mieux admirer la répartition de données

put:http://1.x24.fr/a/jupyter/off/fat-energy-bigDots.webp
put:http://1.x24.fr/a/jupyter/off/fat-energy-tinyDots.webp


'fat-energy-tinyDots.png'

### => Perfectionnisme
- J'ai recherché longtemps à produire des graphs permettant à l'humain lambda de comprendre certains phénomènes
- Le but était de produire une heatmap de nuages de points, et d'affiner ces derniers, en les groupant les uns à côté des autres

- Baisser l'opacité du scatterplot ( nuage de points ) permet également à grande échelle d'affichage de faire apparaitre des "lignes" assez surprenantes entres certains points : notamment dans les corrélations concernant : energy, carbohydrates
+++++++ ===>  http://1.x24.fr/a/jupyter/off/bigCorrelationMapTop50.webp#HeatMapVsScatter

In [ ]:
#repartir le top50
top50=correlations.where(np.triu(np.ones(correlations.shape),k=1).astype(np.bool)).stack().sort_values(ascending=False)[:50]
idx=top50.index;#carbonara
#50*2 => 100 entre axe x et y
j=0;axx=10;axy=10;

heightPerGraph=widthPerGraph=6
f,axes=plt.subplots(axy,axx,figsize=(axy*heightPerGraph,axx*widthPerGraph))
f.patch.set_facecolor('white')
axes=axes.flatten()#pour les parcourir plus facilement :)

j=0;
for i in range(len(top50)):
  x=idx[i][0];
  y=idx[i][1];
  #maxx=df[x].max();maxy=df[y].max();if maxy>maxx: maxx=maxy
  heat(df,x,y,2,axis=axes[j])
  j=j+1;
  scatter(df,x,y,opacity=0.002,color='purple',size=5,axis=axes[j])
  j=j+1;

x='bigCorrelationMapTop50.png'
f.savefig(x, bbox_inches = 'tight')
webp(x)  
show()

put:http://1.x24.fr/a/jupyter/off/bigCorrelationMapTop50.webp


# Hypothèses

In [ ]:
scatter(df,'sodium_100g','salt_100g')#Top évidence
scatter(df,'nutrition-score-fr_100g','nutrition-score-uk_100g',opacity=1)
scatter(df,'energy_100g','carbohydrates_100g',opacity=1)#affirmer:17KJ
scatter(df,'energy_100g','proteins_100g',opacity=1)#affirmer:17KJ
scatter(df,'energy_100g','fat_100g',opacity=1)#affirmer:34KJ et forte contribution
scatter(df,'carbohydrates_100g','sugars_100g',opacity=0.5)#affirmer:34KJ et forte contribution
#kind='scatter','scatter_kws':dict(alpha=0.5)markers="+",

#Joinplot Ajoute le KDE
x='join-carbo-sugars.png'
sns.jointplot(x="carbohydrates_100g", y="sugars_100g", data=df,joint_kws = {'alpha':0.01}).savefig(x, bbox_inches = 'tight');webp(x)
show()#,plot_kws={'alpha':0.05}

#Pairplot + KDE :) .. Si l'on croise toutes les variables, non ?
i='cholesterol_100g';x='pairplot.scatter.kde.'+i+'.png';#,hue='stringCategory'
sns.pairplot(df,kind='scatter',diag_kind='kde',height=graphHeight,aspect=1,x_vars=Cols100g,y_vars=[i],plot_kws={'alpha':0.05,'s':2},markers="+").savefig(x, bbox_inches = 'tight');webp(x)
show()

i='cholesterol_100g';x='pairplot.scatter.interesting.'+i+'.png';#,hue='stringCategory'
j='salt_100g,fiber_100g,proteins_100g,saturated-fat_100g,sugars_100g,additives_n'.split(',')
#diag_kind='kde', ne fonctionne pas sur le pairplot ..
sns.pairplot(df,kind='scatter',height=graphHeight,aspect=1,x_vars=j,y_vars=i,plot_kws={'alpha':0.05,'s':2},markers="+").savefig(x, bbox_inches = 'tight');webp(x);

show()

FPC('allVars',allVars)
ftpput('allVars.pickle')#ajout de variables entre temps ?

put:http://1.x24.fr/a/jupyter/off/sodium_100g.salt_100g.webp
put:http://1.x24.fr/a/jupyter/off/nutrition-score-fr_100g.nutrition-score-uk_100g.webp
put:http://1.x24.fr/a/jupyter/off/energy_100g.carbohydrates_100g.webp
put:http://1.x24.fr/a/jupyter/off/energy_100g.proteins_100g.webp
put:http://1.x24.fr/a/jupyter/off/energy_100g.fat_100g.webp
put:http://1.x24.fr/a/jupyter/off/carbohydrates_100g.sugars_100g.webp
put:http://1.x24.fr/a/jupyter/off/join-carbo-sugars.webp
put:http://1.x24.fr/a/jupyter/off/pairplot.scatter.kde.cholesterol_100g.webp
put:http://1.x24.fr/a/jupyter/off/pairplot.scatter.interesting.cholesterol_100g.webp
put:http://1.x24.fr/a/jupyter/off/allVars.pickle.zip
put:http://1.x24.fr/a/jupyter/off/allVars.pickle.zip


### Corrélations => Conclusions

- Le sel étant une molécule formée à partir de Chlore et Sodium, sa corrélation avec le sodium est absolue, ce qui constitue un fait étalon dans cette matrice de corrélation : avec un indice de 1 on obtient une droite, un indice de -1 et elle sera intersée : Il y a 40g de sodium dans 100g de sel (NaCl, cf : "Pour calculer approximativement la teneur en sodium, on divise la teneur en sel par 2,54") http://1.x24.fr/a/jupyter/off/scatter.sodium_100g.salt_100g.png
<hr>
- Il y a une très forte corrélation entre le nutritionscore uk et fr ( les méthodes de calcul sont très proches ou saisies en doublons ??? ) http://1.x24.fr/a/jupyter/off/scatter.nutrition-score-fr_100g.nutrition-score-uk_100g.png
<hr>

- Certains nutri-élements influent la quantité d'énergie en KJ disponible dans un aliment : on observe un pic à 1700KJ pour 100% de glucides ou protéines, puis 3700KJ pour les lipides
<br>=> On peut affirmer que les glucides et protéines apportent 17KJ par gramme, 37KJ pour les lipides
http://1.x24.fr/a/jupyter/off/scatter.energy_100g.carbohydrates_100g.png http://1.x24.fr/a/jupyter/off/scatter.energy_100g.proteins_100g.png http://1.x24.fr/a/jupyter/off/scatter.energy_100g.fat_100g.png
<hr>
- Les carbohydrates sont les glucides et regroupent donc tous les sucres, il est donc normal qu'il y en ait autant que de sucres cumulés, et sa teneur est inversement proportionnelle à la proportion d'autres nutriments !
- La courbe de corrélation avec l'énergie est très intéressante vu que l'on constate un pic de 100% autour de 1700kj ( 17kj par gramme de sucre ) ce qui confirme la pertinence des données
http://1.x24.fr/a/jupyter/off/join-carbo-sugars.png
<hr>
- On trouve plus cholestérol dans les aliments<20% de sel ( et sodium ), <40% fibres , <40% protéines, <40% graisses saturées, plus il y a de sucres ou d'additifs le taux semble baisser
http://1.x24.fr/a/jupyter/off/pairplot.scatter.interesting.cholesterol_100g.png http://1.x24.fr/a/jupyter/off/pairplot.scatter.interesting.cholesterol_100g.png
<hr>
- Le score nutriotion est plus elevé plus il y a de grassises saturées, de gras et que l'aliment est calorique
- Plus de sel ( sodium et sel ) dans les aliments protéinés
<hr>
<sep/>

In [ ]:
#On se crée le display de toutes les colonnes pour ensuite sélectionner ce qui semble nous intéresser en premier lieu
columns=df.columns;
print(','.join(columns));

relevancy,notes,sum,kj,code,creator,product_name,quantity,packaging,brands,categories,categories_en,origins,manufacturing_places,labels,labels_tags,labels_en,purchase_places,stores,countries_en,additives_n,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,main_category,main_category_en,energy_100g,fat_100g,saturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,nutrition-score-fr_100g,nutrition-score-uk_100g


## Multivariée
=> Créer de nouvelles colonnes composites ( sommes d'autres, dummies from strings, concaténations de chaines de caractères ??? )
- => Il suffit de consulter la heatmap des corrélations : http://1.x24.fr/a/jupyter/off/off-AllCorrelations.png, la présence d'élément apportant des calories à un aliment contribuent à son apport énergétique
- Si les variables sont numériques alors il faut privilégier un plot 3D ou interactif : plotly express

### Affirmation

In [ ]:
#resume();
#type(sup0)==pandas.core.series.Series
x='fat_100g';y='carbohydrates_100g';z='energy_100g';z2='proteins_100g'
xmin=1;xmax=101;ymin=1;ymax=101;zmin=1;zmax=3800
colors=px.colors.diverging.RdYlBu;#'Magma_r'#px.colors.sequential.Viridis,px.colors.diverging.RdYlBu,Tealrose_r)

df['color']=(df[z]/(df[z].max()/100)).astype('int')#Avoir 100 Couleurs max
df['symbol']=(df[z2]/(df[z2].max()/20)).astype('int')#20 symbols pour les niveaux différents de protéines, or country classification
#df is always a Positive value !!!!
df['size']=(df[z2]/(df[z2].max()/20)).astype('int')+2
df['size'].min()

#rug=fringes, box=mustaches, violin, histogram
#ValueError: Format 'energy_100g' is not supported (supported formats: eps, jpeg, jpg, pdf, pgf, png, ps, raw, rgba, svg, svgz, tif, tiff)
scatter(df,'fat_100g','energy_100g')#joinplot with kde
scatter(df,'carbohydrates_100g','energy_100g')
scatter(df,'carbohydrates_100g','fat_100g')

put:http://1.x24.fr/a/jupyter/off/fat_100g.energy_100g.webp
put:http://1.x24.fr/a/jupyter/off/carbohydrates_100g.energy_100g.webp
put:http://1.x24.fr/a/jupyter/off/carbohydrates_100g.fat_100g.webp


1

### Graphes avec courbe de regression

In [ ]:
samples=2000;samples=400000;

snsscat(df[(df['fat_100g']>1) & (df['energy_100g']>1)][:samples],'fat_100g','energy_100g',opacity=0.01,color=[1,0,0.3,0.01],kind='reg',fn='fat-energy-reg.png')#Reg
snsscat(df[(df['sodium_100g']>1) & (df['salt_100g']>1)][:samples],'sodium_100g','salt_100g',opacity=0.1,color=[1,0,0.3,0.1],kind='reg',fn='sodium-salt-reg.png')#Reg
snsscat(df[(df['nutrition-score-fr_100g']>1) & (df['nutrition-score-uk_100g']>1)][:samples],'nutrition-score-fr_100g','nutrition-score-uk_100g',opacity=0.1,color=[1,0,0.3,0.1],kind='reg',fn='nutrition-reg.png')#Reg
snsscat(df[(df['carbohydrates_100g']>1) & (df['energy_100g']>1)][:samples],'carbohydrates_100g','energy_100g',opacity=0.01,color=[1,0,1,0.01],kind='reg',fn='carbo-energy-reg.png')#Reg
snsscat(df[(df['proteins_100g']>1) & (df['energy_100g']>1)][:samples],'proteins_100g','energy_100g',opacity=0.01,color=[0,1,1,0.01],kind='reg',fn='proteins_100g-energy-reg.png')#Reg
#plt.offline.plot(ax, filename='plotlyExpressViolins-carbovsFat.html')df.columns
#seaborn boxplot multiple sub - categories
#hue=variable à 9 dimensions subdivisant l'ensemble ?
snsscat(df[(df['fat_100g']>1) & (df['energy_100g']>1)][:samples],'fat_100g','energy_100g',opacity=1,color=[1,0,0.4],kind='hex',fn='fat-energy-hex.png')#Hex: Raise Opacity
snsscat(df[(df['fat_100g']>1) & (df['energy_100g']>1)][:samples],'fat_100g','energy_100g',opacity=0.01,color=[1,0,0.3,0.01],kind='kde',fn='fat-energy-kde.png')#kde²=contour - pic 1500=0%fat 100%sucres
#snsscat(df[(df['fat_100g']>1) & (df['energy_100g']>1)][:samples],'fat_100g','energy_100g',opacity=0.01,color=[1,0,0.3,0.01],fn='fat-energy-scat.png')#SCate
#g = (sns.jointplot("sepal_length", "sepal_width",data=iris, color="k").plot_joint(sns.kdeplot, zorder=0, n_levels=9))  #contour over scatter kde
#distributions inversement proportionnelles
snsscat(df[(df['fat_100g']>1) & (df['carbohydrates_100g']>1)][:samples],'fat_100g','carbohydrates_100g',opacity=0.01,color='red',fn='fatVsCarbo.png')
snsscat(df[(df['fat_100g']>1) & (df['proteins_100g']>1)][:samples],'fat_100g','proteins_100g',opacity=0.1,color='red',fn='fatVsProteins.png')
#python function apply dict parameters

/usr/local/lib/python3.6/dist-packages/seaborn/axisgrid.py:1847: UserWarning:

JointGrid annotation is deprecated and will be removed in a future release.



put:http://1.x24.fr/a/jupyter/off/fat-energy-reg.webp
put:http://1.x24.fr/a/jupyter/off/sodium-salt-reg.webp
put:http://1.x24.fr/a/jupyter/off/nutrition-reg.webp
put:http://1.x24.fr/a/jupyter/off/carbo-energy-reg.webp
put:http://1.x24.fr/a/jupyter/off/proteins_100g-energy-reg.webp
put:http://1.x24.fr/a/jupyter/off/fat-energy-hex.webp
put:http://1.x24.fr/a/jupyter/off/fat-energy-kde.webp
put:http://1.x24.fr/a/jupyter/off/fatVsCarbo.webp
put:http://1.x24.fr/a/jupyter/off/fatVsProteins.webp


1

### Px Express Violins

In [ ]:
%%script false 
samples=1000
ax=px.scatter(df[:samples], x=x, y=y, marginal_y="rug", marginal_x="histogram" ,opacity=1, color='color', symbol='symbol',color_continuous_scale=colors, trendline="ols")
ax.update_layout(scene=dict(xaxis_title=x,yaxis_title=y,xaxis=dict(nticks=4,range=[xmin,xmax],),yaxis=dict(nticks=4,range=[ymin,ymax],),zaxis = dict(range=[zmin,zmax],),));#zaxis_limit=3700,
ax.show()

#size='size', 
ax=px.scatter(df[:samples], x=x, y=y, marginal_y="violin", marginal_x="violin" ,opacity=1, color='color', symbol='symbol',color_continuous_scale=colors, trendline="ols")
ax.update_layout(scene=dict(xaxis_title=x,yaxis_title=y,xaxis=dict(nticks=4,range=[xmin,xmax],),yaxis=dict(nticks=4,range=[ymin,ymax],),zaxis = dict(range=[zmin,zmax],),));#zaxis_limit=3700,
ax.show()

px.scatter(df[:samples], x=x, y=y, marginal_y="violin",marginal_x="box", trendline="ols").show()#SimpleCarboOnFats

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.



### 3d

In [ ]:
%%script false 
#rendering takes 8mo
df['size']=(df['fiber_100g']/5).astype('int')#0 à 20  
#df2['symbol']=(df2['fiber_100g']/10).astype('int')#12 Valeurs distincte, plus selon une catégorisation, non ?
df['symbol']=1#toujours un rond, on pourrait utiliser une classification à cet emplacement
color='proteins_100g';size='size';symbol='symbol'

def d3(df2,x,y,z,o,dfportion=1000000,color=None,size=None,symbol=None,xmin=1,xmax=101,ymin=1,ymax=101,zmin=0,zmax=3800):
#hover_data=tips.columns
#plotly-express #dir(px.scatter_3d)
  colors=px.colors.sequential.Viridis#px.colors.diverging.RdYlGn;#RdYlBu
#aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance','blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg','brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl','darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric','emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys','haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet','magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges','orrd', 'oryel', 'peach', 'phase', 'picnic', 'pinkyl', 'piyg','plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn', 'puor','purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu', 'rdgy','rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar', 'spectral','speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn', 'tealrose','tempo', 'temps', 'thermal', 'tropic', 'turbid', 'twilight','viridis', 'ylgn', 'ylgnbu', 'ylorbr', 'ylorrd'  
  fig=px.scatter_3d(df2[:dfportion],x=x,y=y,z=z,color=color,size=size,symbol=symbol,size_max=100,opacity=o,color_continuous_scale='hsv')#,size=abs(ColumnName), color='species', symbol='species',animation_frame='date or yearColumn'
  fig.update_traces(marker=dict(line=dict(width=1,color=[0,0,0,o])),selector=dict(mode='markers'))#,alpha=o
  fig.update_layout(scene=dict(xaxis_title=x,yaxis_title=y,zaxis_title=z,xaxis=dict(nticks=4,range=[xmin,xmax],),yaxis=dict(nticks=4,range=[ymin,ymax],),zaxis = dict(range=[zmin,zmax],),));#zaxis_limit=3700,
  fig.update_xaxes(ticks="inside", tickwidth=2, tickcolor='crimson', ticklen=10,col=1)#showgrid=True, gridwidth=1, gridcolor='LightPink'
  fig.show()
#plotly-express color_continuous_scale
#créer de nouvelles colonnes afin de mettre en relief certaines données
#grouped class value for symbol : otherwise, take lot of time !
d3(df,x,y,z,o=0.003,color=color)#tous without fancy stuff, todo:modifier l'échelle des chaleurs
d3(df,x,y,z,o=0.1,dfportion=5000,color=color)#tous without fancy stuff, todo:modifier l'échelle des chaleurs, plus de points, plus de taille
#TODO::::scatter(df2,df2[x],df2[y])
#d3(df2,x,y,z,0.1,10000,color,size,symbol)#moins d'échantillons avec plus d'opactié
#d3(df2,x,y,z,0.001,360000,color,size,symbol)#tous

### Contour
 - http://1.x24.fr/a/jupyter/off/3.contour-energy_100g-fat_100g-carbohydrates_100g-20-400000.webp

In [ ]:
%%script false
#restore()
#http://1.x24.fr/a/jupyter/off/3.contour-energy_100g-fat_100g-carbohydrates_100g-20-400000.webp
def contour(data,x,y,z,fn=0):
  from scipy.interpolate import griddata
  df=data.copy(deep=True)#&
  if(fn==0):
    fn='3.contour-'+z+'-'+x+'-'+y+'-'+str(levels)+'-'+str(limit)+'.png'
  #data Cleanup
  for i in [x,y,z]:
    if(df.dtypes[i] != object):#integer values only
      df.loc[np.isnan(df[i]),i]=0
      df.loc[(df[i]<0),i]=0 
      df.loc[(df[i]>getLimit(i)),i]=getLimit(i)

  x0=df[x];y0=df[y];z0=df[z]
  plt.title(z+' / '+x+' & '+y);plt.xlabel(x);plt.ylabel(y)   
#CS=plt.tricontour(x0,y0,z0,levels)
  x1 = np.linspace(x0.min(), x0.max(), len(x0.unique()))#pour autant de valeurs uniques, espacer et créer le contour
  y1 = np.linspace(y0.min(), y0.max(), len(y0.unique()))
  x2, y2 = np.meshgrid(x1, y1)
  #z2 = griddata(x, y, z, xi, yi, interp='linear')
  z2 = griddata((x0, y0), z0, (x2, y2), method='linear')#, method='cubic' => throws out of bound values
  #Xflat, Yflat, Zflat = x2.flatten(), y2.flatten(), z2.flatten()
  #plt.contourf(x2,y2,z2)
  #plt.zlabel('energy_100g')
  #fig = plt.figure()
  #ax = fig.gca()
  #ax.set_zlim(0,getLimit(z))
  CS = plt.contourf(x2, y2, z2,levels, vmin=0, vmax=getLimit(z))#First,zlim=getLimit(z)
  #plt.clim(0,getLimit(z))#otherwise strange negative value will show up
  #plt.scatter(x0,y0,alpha=200/limit,s=2)
  CS2 = plt.contour(CS, levels=CS.levels[::2],colors='black',alpha=0.1)#contour tous les 2 niveaux, mais si ils sont trop densément rapprochés à certains endroits .. ??
  cbar = plt.colorbar(CS);cbar.ax.set_ylabel(z);cbar.add_lines(CS2);#Legend
  plt.tight_layout()
  plt.savefig(fn, bbox_inches='tight')
  webp(fn) 
  show()

x='fat_100g';y='carbohydrates_100g';z='energy_100g';
levels=20;
limit=400000;
heightPerGraph=widthPerGraph=24
#fn='3.contour-energy_100g-fat_100g-carbohydrates_100g-20-400000.webp'
plt.rcParams["figure.figsize"] = (widthPerGraph,heightPerGraph)
#Filtrer les résultats abérrants
# & (df['energy_100g']<3900) & (df['energy_100g']>0)
dfLimited=df[(df['sum']<100)][:limit]
contour(dfLimited,x,y,z)

put:http://1.x24.fr/a/jupyter/off/3.contour-energy_100g-fat_100g-carbohydrates_100g-20-400000.webp


### Par catégories
- Diagnostic colonnes uniques de types strings, puis selection des 30 premières pour tracer des graphs pairplot pour chacun des nutri-éléments

In [ ]:
#resume()categories_en,main_category_en,pnns_groups_2,pnns_groups_1,nutrition_grade_fr,stores   
#df4=loadData(fn)allVars
columns=df.columns
#f, ax = plt.subplots(figsize=(50,40))
plt.rcParams["figure.figsize"] = (12,24)
sns.set_style("darkgrid")#, {"axes.facecolor": ".9"}

nbTopValues=51

#1ère dimension:countries
a='countries_en'
df[a]=df[a].apply(lowerNoSpace)
topCountries=uniqueValuesPerColumn(a,df)[:10]
exclusions='code,url,revelancy,notes,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_tags'.split(',')
#product_name
#!rm 70-violin*
#seconde dimension
#df.select_dtypes(include=['object']).dtypes
#df.dtypes['object']
for i in df.dtypes.index:#str(df4.dtypes['code'])  
  if(i in exclusions):
    continue;
  if((str(df.dtypes[i])=='object')):    
    x='70-violin-TopCountriesHue-'+i+'-energy_100g.png' 
    if(os.path.exists(x)==True):
      continue;
    print('_'*140)
    print(i)
    #i='categories_en';#Harmonization : no space to lower case
    filtered=df.copy(deep=True)#Reset, always, ne pas altérer les passages en référence
    filtered[i]=df[i].apply(lastCommaValue)#gets last category out of tree
    filtered[i]=filtered[i].apply(lowerNoSpace)#harmonize records
    topValues=uniqueValuesPerColumn(i,filtered)[:nbTopValues]#get the top values
    filtered=filtered[filtered[i].isin(topValues.index) & filtered[i]!=0 & (filtered[i]!='0')]#exclude 0 values -1
#=> python dataframe sort by column frequency ::  shttps://stackoverflow.com/questions/35036182/sort-a-dataframe-column-by-the-frequency-of-occurrence/35036797
    sortedDataframe=filtered.assign(freq=filtered.groupby(i)[i].transform('count'))###.loc[:,[i]]
    
#de bas en haut pour les box, gauche à droite pour les violins       
    violins=sortedDataframe.sort_values(by=['freq',i],ascending=[False,True])
    if(i!=a):#not the first dimension
      violinsTopCountries=violins[violins[a].isin(topCountries.index)]
  #0-violin-quantity-energy_100g.png
      plt.rcParams["figure.figsize"] = (300,12); 
      #ValueError: min() arg is an empty sequence after additives_en
      g=sns.violinplot(hue=violinsTopCountries[a],y=violinsTopCountries['energy_100g'],x=violinsTopCountries[i],scale='width',height=40,gridspec_kws={'wspace':.03});g.set_xticklabels(g.get_xticklabels(), rotation=90, ha="right");g.get_figure().savefig(x,bbox_inches='tight');ftpput(x);show()

    plt.rcParams["figure.figsize"] = (120,12); 
    x='70-violin-area-'+i+'-energy_100g.png'
    g=sns.violinplot(scale='area',y=violins['energy_100g'],x=violins[i],height=40,gridspec_kws={'wspace':.03});g.set_xticklabels(g.get_xticklabels(), rotation=90, ha="right");g.get_figure().savefig(x,bbox_inches='tight');ftpput(x);show()
    x='70-violin-width-'+i+'-energy_100g.png'
    g=sns.violinplot(scale='width',y=violins['energy_100g'],x=violins[i],height=40,gridspec_kws={'wspace':.03});g.set_xticklabels(g.get_xticklabels(), rotation=90, ha="right");g.get_figure().savefig(x,bbox_inches='tight');ftpput(x);show()
    x='70-violin-count-'+i+'-energy_100g.png'
    g=sns.violinplot(scale='count',y=violins['energy_100g'],x=violins[i],height=40,gridspec_kws={'wspace':.03});g.set_xticklabels(g.get_xticklabels(), rotation=90, ha="right");g.get_figure().savefig(x,bbox_inches='tight');ftpput(x);show()
#d.DataFrame({col:vals['absolute_error'] for col,vals in grouped}).mean().sort_values(ascending=True)    
    #pairplot y_vars order, Première catégorie donnée en bas, donc on inverse l'order
    #sns.pairplot(data=sortedDataframe,height=10,aspect=1,kind='scatter',plot_kws={'alpha':0.3},x_vars=Cols100g,y_vars=[i]).savefig(x,bbox_inches='tight');

#de bas en haut pour les box, gauche à droite pour les violins       
    box=sortedDataframe.sort_values(by=['freq',i],ascending=[True,True])    
    x='boxplot-'+i+'-energy_100g.png'
    plt.rcParams["figure.figsize"] = (12,24)
    g=v#,,linewidth=1,yticklabels=True
    g.set_yticklabels(g.get_yticklabels(), rotation=40, ha="right")
    g.get_figure().savefig(x,bbox_inches='tight');webp(x);
    show()
    continue;
#multivariée : pairplot ? => Confus et peu utiles
    x='70-pairplot-'+i+'-cols100g.png'
    sns.pairplot(data=sortedDataframe,height=10,aspect=1,x_vars=Cols100g,y_vars=[i],kind='scatter').savefig(x,bbox_inches='tight');webp(x);show()
    continue;    
    #for j in top30:
    #x='70-violin-'+i+'-energy_100g.png'
    #g=sns.pairplot(filtered,kind='scatter',x_vars=[i],y_vars=Cols100g,plot_kws={'alpha':0.2},markers="+").savefig(x,bbox_inches='tight');ftpput(x);show()#width=height*aspect,hue='LargestPropertyUseType'#,y_vars
    #assert(False)
#
if False:
  i='categories_en'    
  top50=uniqueValuesPerColumn(i,df)[:51]
  filtered=df[df[i].isin(top50.index) & df[i]!=0 & (df[i]!='0')]#-1
  filtered[i]=filtered[i].apply(lastCat)         

  #subplots_adjust  
  plt.tight_layout()
  #, inner="stick",order = sorted(df.Category.unique()),scale="count",split=True,split=True,scale_hue=False

  sns.violinplot(y=sortedDataframe['energy_100g'],x=sortedDataframe[i],height=40,gridspec_kws={'hspace':.01},bw=.2).get_figure().savefig(x,bbox_inches='tight');webp(x);show()

____________________________________________________________________________________________________________________________________________
creator
put:http://1.x24.fr/a/jupyter/off/70-violin-TopCountriesHue-creator-energy_100g.png
put:http://1.x24.fr/a/jupyter/off/70-violin-area-creator-energy_100g.png
put:http://1.x24.fr/a/jupyter/off/70-violin-width-creator-energy_100g.png
put:http://1.x24.fr/a/jupyter/off/70-violin-count-creator-energy_100g.png
put:http://1.x24.fr/a/jupyter/off/boxplot-creator-energy_100g.png
____________________________________________________________________________________________________________________________________________
product_name
put:http://1.x24.fr/a/jupyter/off/70-violin-TopCountriesHue-product_name-energy_100g.png
put:http://1.x24.fr/a/jupyter/off/70-violin-area-product_name-energy_100g.png
put:http://1.x24.fr/a/jupyter/off/70-violin-width-product_name-energy_100g.png
put:http://1.x24.fr/a/jupyter/off/70-violin-count-product_name-energy_100g.png
p

3) Conclusions
<br>
-  Par Nom de produit : les haricots vers sont moins caloriques que l'huile d'olive ou mayonnaise
http://1.x24.fr/a/jupyter/off/boxplot-product_name-energy_100g.png

- Apperizers>Vegetables
http://1.x24.fr/a/jupyter/off/boxplot-pnns_groups_2-energy_100g.png

-  SaltySnacks> Vegetables
http://1.x24.fr/a/jupyter/off/boxplot-pnns_groups_2-energy_100g.png

-  ???
http://1.x24.fr/a/jupyter/off/boxplot-main_category_en-energy_100g.png

-  ???
http://1.x24.fr/a/jupyter/off/boxplot-brands-energy_100g.png

-  Plus grand contributeurs
http://1.x24.fr/a/jupyter/off/boxplot-creator-energy_100g.png

-  Par catégories de produits : l'huile d'olive est bien plus calorique que les boissons non sucrées
http://1.x24.fr/a/jupyter/off/boxplot-categories_en-energy_100g.png

-  Teneurs énergétiques par pays en bas  les catégories les plus représentées
http://1.x24.fr/a/jupyter/off/boxplot-countries_en-energy_100g.png

-  Par magasin
http://1.x24.fr/a/jupyter/off/boxplot-stores-energy_100g.png

-  Les produits provenant de chine sont bien moins caloriques que ceux venant du sud ouest .. ( foie gras, sarladaise )
http://1.x24.fr/a/jupyter/off/boxplot-origins-energy_100g.png


#Statistiques

## Statistiques croisées
- On désire croiser des stats entre des variables x ( strings ) et des variables y ( numériques ) afin de déceler de potentielles anomalies ..

In [ ]:
#original Non Cleaned Dataframe
df4=loadData(fn);
df4.insert(0,"relevancy",100) 
columns=list(df4.columns.values)

x_stats='countries_en,cities,origins,brands,product_name'.split(',');
#nutrition_grade_fr,ingredients_from_palm_oil,
y_stats='energy_100g,energy-from-fat_100g,omega-3-fat_100g,carbohydrates_100g,cholesterol_100g,salt_100g,alcohol_100g,fiber_100g,proteins_100g,carbon-footprint_100g,nutrition-score-fr_100g'.split(',');
#df['nutrition_grade_fr'].describe();#nutrition_grade_fr, Length: 356027, dtype: object
debug;

## Corrélations rapides
- sous forme de tableau http://1.x24.fr/a/jupyter/off/90-heatmap-big-AllVariables.png


In [ ]:
#python array is dict or list
fitness=x_stats[:1];#country_en
if'the big heatmap':
  heatmap=df4.copy(deep=True);
  for i in df4.dtypes.index:
    if(df4.dtypes[i]=='object'):
      heatmap[i],levels=pd.factorize(heatmap[i])

  correlations=heatmap.corr(method='pearson');      
  np.fill_diagonal(correlations.values, np.nan)#no more ones diagonal
  c=correlations.shape[1];

  plt.figure(figsize=(c,c));
  x='90-heatmap-big-AllVariables.png'
  sns.heatmap(correlations,xticklabels=correlations.columns.values,yticklabels=correlations.columns.values,annot=True,cmap=sns.diverging_palette(220, 10, as_cmap=True)).get_figure().savefig(x,bbox_inches='tight');webp(x);show()

heatmap=df4[x_stats+y_stats].copy(deep=True);
#correlations = heatmap.loc[:,heatmap.dtypes == 'float64'].corr().abs();#Negative Value is a correlation, but acts most of the times negatively : ex Pclass 3 Acts on Survival Rate => Death ++


for i in heatmap.columns.values:
  #convert all data 
  if 'numpy' not in str(type(heatmap[i][0])): 
    heatmap[i],levels=pd.factorize(heatmap[i])
        
correlations=heatmap.corr().abs();#Negative Value is a correlation, but acts most of the times negatively : ex Pclass 3 Acts on Survival Rate => Death ++        
topCorrelations=(correlations.where(np.triu(np.ones(correlations.shape), k=1).astype(np.bool)).stack().sort_values(ascending=False))[:40]
topCorrelations

put:http://1.x24.fr/a/jupyter/off/90-heatmap-big-AllVariables.png


carbohydrates_100g     proteins_100g              1.000000
energy_100g            energy-from-fat_100g       0.769046
brands                 product_name               0.611664
energy-from-fat_100g   nutrition-score-fr_100g    0.543250
omega-3-fat_100g       carbon-footprint_100g      0.542387
energy_100g            nutrition-score-fr_100g    0.533250
                       carbohydrates_100g         0.469803
carbon-footprint_100g  nutrition-score-fr_100g    0.375969
proteins_100g          carbon-footprint_100g      0.365731
energy_100g            omega-3-fat_100g           0.352306
                       proteins_100g              0.272950
salt_100g              carbon-footprint_100g      0.262297
carbohydrates_100g     nutrition-score-fr_100g    0.254525
energy-from-fat_100g   cholesterol_100g           0.217226
energy_100g            carbon-footprint_100g      0.185085
omega-3-fat_100g       fiber_100g                 0.178136
                       nutrition-score-fr_100g    0.1760

## Constat corrélations
- On constate des corrélations évidentes entre les valeurs textes : marques, nom de produit et pays d'origine
- On constate corrélation entre teneur en (omega_3,protéines,score de nutrition, sel) pour l'emprunte carbone ( qui est très rarement remplie, uniquement en France et en Suisse )
- Le score de nutrition a l'air très lié à l'energie disponible depuis le gras, soit la teneur en gras évidente
- La teneur en omega3 est liée à la teneur en graisse vu que ce sont des lipides
- Même constats évident pour le cholestérol etc ..

In [ ]:
if'shortheatmaps':
  j=1;r=1;c=correlations.shape[1];#one row minimalist per index
  for i in correlations.index:
    if(i=='ingrediens_from_palm_oil'):
      break;
    if(i in 'relevancy,score,cities,allergens_en'.split(',')):
      j+=1;
      continue  
    #i='energy_100g';print(i)
    plt.figure(figsize=(c,r));
    x='90-heatmap-short-'+i+'.png'
    #correlations[correlations[i]],correlations.ix[i]
    #ValueError: zero-size array to reduction operation minimum which has no identity:cities
    sns.heatmap(correlations.iloc[j-1:j],xticklabels=correlations.columns.values,yticklabels=[i],annot=True,cmap="YlGnBu").get_figure().savefig(x,bbox_inches='tight');webp(x);show();j+=1

plt.figure(figsize=(c,c));
x='90-heatmap-big-basic.png'
sns.heatmap(correlations,xticklabels=correlations.columns.values,yticklabels=correlations.columns.values,annot=True,cmap=sns.diverging_palette(220, 10, as_cmap=True)).get_figure().savefig(x,bbox_inches='tight');
webp(x);show()
#plt.figure(figsize=(c,c));sns.heatmap(correlations,xticklabels=correlations.columns.values,yticklabels=correlations.columns.values,annot=True,cmap="YlGnBu");#minimalistic heatmap 1 Row

droppedColumns=list(set(heatmap.columns.values) - set(correlations.columns.values))
#print('droppedColumns');print(droppedColumns);#Si jamais les valeurs ne sont pas toutes factorisées ..
#dropped=c1-c;print(str(dropped)+' columns ignored..')
#numpt.diff(heatmap)
#Brands and product names are most likely to correlate with a given country name

#minimalistic heatmap 1 Row
#Brands are highly correlated with their products name
#Energy is 77% linked with energy from fat( litteraly fat value), omega3, carbonydrates(sugar)
#omega3, proteins, salt have an impact on co2 footprint ..


put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-countries_en.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-origins.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-brands.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-product_name.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-energy_100g.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-energy-from-fat_100g.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-omega-3-fat_100g.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-carbohydrates_100g.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-cholesterol_100g.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-salt_100g.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-alcohol_100g.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-fiber_100g.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-proteins_100g.png
put:http://1.x24.fr/a/jupyter/off/90-heatmap-short-carbon-footprint_100g.png
put:http://1.x24.fr/a/jupyter/off

#### Par pays
Constater à quel point certaines données manquent, notamment pour :
- carbon-footprint_100g
- omega3_in_fat

In [ ]:
#Per Column stats : dditives_n,ingredients_that_may_be_from_palm_oil_n,
#if __name__ == "__main__":
#columns=list(df4.columns.values)
for y in columns:
    if('numpy' not in str(type(df4[y][0]))):
        continue;
    print('_'*60);
    print(y)
    #[(df4['relevancy']>acceptableScore)]
    print(df4.groupby('countries_en')[y].agg(['count','mean']).sort_values(by='count',ascending=False)[:3]);

____________________________________________________________
relevancy
                count  mean
countries_en               
United States  173159   100
France         123961   100
Switzerland     14932   100
____________________________________________________________
no_nutriments
                                           count  mean
countries_en                                          
Afghanistan                                    0   NaN
Germany,Hungary                                0   NaN
France,fr:Bourgogne-aube-nogent-sur-seine      0   NaN
____________________________________________________________
additives_n
                count      mean
countries_en                   
United States  171521  2.055200
France          82338  1.700345
Switzerland      7902  1.666034
____________________________________________________________
ingredients_from_palm_oil_n
                count      mean
countries_en                   
United States  171521  0.000012
France          82338

### Par colonne, occurences
- Ce qui permet de repérer les variables très fortement corréllées qui ont peu d'enregistrements ..

In [ ]:
#Grouped mean Values for y in all x axis - in order to get anomalies pretty fast    
for x in x_stats:
    for y in y_stats:
      #[(df['relevancy']>acceptableScore)]
        s=df4.groupby(x)[y].mean().sort_values(ascending=False);#DataError: No numeric types to aggregate:nutrition_grade_fr, dtype: int64
        #todo:s.plot
        print(y);
        print(s.head(2));print('_'*60);
    break;

energy_100g
countries_en
Czech Republic,Germany       3443.0
Croatia,Montenegro,Serbia    3099.0
Name: energy_100g, dtype: float64
____________________________________________________________
energy-from-fat_100g
countries_en
Canada,France,Switzerland,United States    1240.0
Canada,United States                       1140.0
Name: energy-from-fat_100g, dtype: float64
____________________________________________________________
omega-3-fat_100g
countries_en
France,Portugal    31.0
France,Réunion     22.5
Name: omega-3-fat_100g, dtype: float64
____________________________________________________________
carbohydrates_100g
countries_en
China                    1.044444e+09
Denmark,Norway,Sweden    9.900000e+01
Name: carbohydrates_100g, dtype: float64
____________________________________________________________
cholesterol_100g
countries_en
Belgium,France,Switzerland      0.378
France,Germany,United States    0.073
Name: cholesterol_100g, dtype: float64
_____________________________________

#Par pays 

## Pays mals ordonnés
- On a constaté que la plupart des pays ne sont pas uniques ( écrits dans n langues respectives ), ou contenaient des erreurs, ou sont une concaténation de pays ( produits transformés ?? )

In [ ]:
# remplir les valeurs nulles#charger dataframe originel
#resume()
#pd.concat(frames)#rows
if False:
  z=allVars['valeursExotiques']
  df=allVars['df']
  #Nettoyage
  for i in z.columns.values:
    z.loc[np.isnan(z[i]),i]=0
    z.loc[(z[i]<0),i]=0  
    z.loc[(z[i]>getLimit(i)),i]=getLimit(i)
  #df['fiber_100g'].max()
  #not in Cols100g retenues
  for i in df.columns.values:
    if(str(df.dtypes[i])!='object'):  
      if(('nutri' not in i) & ('revelancy' not in i) & ('score' not in i)):
        df.loc[(df[i]<0),i]=0  
      df.loc[np.isnan(df[i]),i]=0    
      df.loc[(df[i]>getLimit(i)),i]=getLimit(i)

  df4=pd.concat([df,z], axis=1)#Ré-inclure les valeurs "exotiques"
  x='recombined.tsv'
  df4.to_csv(x,sep='\t');
  ftpput(x)

for i in df4.columns.values:
  if(str(df4.dtypes[i])!='object'):  
    if(('nutri' not in i) & ('revelancy' not in i) & ('score' not in i)):
      df4.loc[(df4[i]<0),i]=0  
    df4.loc[np.isnan(df4[i]),i]=0    
    df4.loc[(df4[i]>getLimit(i)),i]=getLimit(i)
    
df4['countries_en'].fillna('na',inplace=True);

Countries = {}
CountriesList = []
#obtention des valeurs uniques
for country in list(df4['countries_en'].unique()):
    try:
        if len(country.split(',')) == 1:
            #concaténation
            CountriesList.append(country)
    except:
        print('Error:', country)
        

#Pour chaque pays extrait, on lui assigne toutes les valeurs uniques ( des traductions associées )
for country in CountriesList:
    Countries[country] = list(df4[df4['countries_en'] == country]['countries'].unique())#unique values for this
    
#Beware as some country values are not relevant at all "en:fresh-foods","fr:Demonte-italie" ISO country Code is better
print('Nb of countries : '+str(len(Countries)));
Countries

#for i in nutriments:#

Nb of countries : 185


{'Afghanistan': ['en:AF'],
 'Albania': ['en:AL', 'Albania'],
 'Algeria': ['en:DZ', 'Algérie', 'Algeria', 'الجزائر', 'algeria'],
 'Andorra': ['en:AD', 'Andorra', 'Andorre'],
 'Angola': ['en:AO', 'Angola'],
 'Ap': ['en:AP'],
 'Argentina': ['en:AR', 'Argentina', 'argentina'],
 'Armenia': ['Arménie', 'en:AM', 'Հայաստան', 'Armenia'],
 'Aruba': ['en:AW', 'Aruba'],
 'Australia': ['en:AU', 'Australia', 'australia', 'Australie'],
 'Austria': ['Österreich', 'en:AT', 'Austria', 'Autriche'],
 'Azerbaijan': ['en:AZ', 'Azerbaijan'],
 'Bahrain': ['en:BH'],
 'Bangladesh': ['en:BD', 'বাংলাদেশ'],
 'Barbados': ['en:BB'],
 'Belarus': ['en:BY', 'Белоруссия', 'Belarus'],
 'Belgie': ['België'],
 'Belgium': ['en:BE',
  'Belgique',
  'Belgium',
  'belgium',
  'België',
  'Бельгия',
  'Belgica',
  'Belgien'],
 'Bolivia': ['en:BO'],
 'Bosnia and Herzegovina': ['en:BA'],
 'Brazil': ['en:BR', 'Brasil', 'Brésil', 'Brazil', 'brazil', 'en:brazil'],
 'Brunei': ['en:BN'],
 'Bulgaria': ['en:BG', 'Bulgaria', 'Bulgarie'],

### GroupBy Pays
- On parcours notre liste de pays pour créer un dictionnaire des valeurs énergétiques moyennes par pays

In [ ]:
#on crée une liste des nutriments qui nous intéressent, puis on les classifie par pays
country2nutriments = {}
debug['nullCountries']=[]
debug['nullCountriesNutriments']=[]

#QUELLES SONT LES UNITES DE MESURE ???
#,nutrition-score-fr_100g'.split(',');energy-from-fat,

nutriments = 'energy,fat,proteins,carbohydrates,sugars,omega-3-fat,cholesterol,salt,alcohol,fiber,carbon-footprint'.split(',')

for country in list(Countries.keys()):
    country2nutriments[country]={};#creates key
    for nutriment in nutriments:
        oob=0        
        x=df4[(df4['countries_en'] == country) & (df4['relevancy']>acceptableScore) & (df4[nutriment + '_100g']>0)][(nutriment + '_100g')]
        total=x.size
        notnull = list(x[x.notnull()])
        ok = len(notnull)
        if (ok == 0):
#No Moldavian Sugar !
            debug['nullCountriesNutriments'].append(country+' '+nutriment);
            continue;#dont need any further processing
        ko = len(x[x.isnull()])        
        mean=round(np.mean(notnull),2);            
        country2nutriments[country][nutriment]={'mean':mean,'ko':ko,'ok':ok,'per':ok/total,'oob':oob}
        
    if(bool(country2nutriments[country])==False):
        del country2nutriments[country];
        debug['nullCountries'].append(country)
        
p('Countries Having no correct nutriments data at all : \n\n'+',\t'.join(debug['nullCountries']))
p('Nb of countries with correct nutriments data : '+str(len(debug['nullCountriesNutriments'])))
p('Nutriments boundaries processed');
#libérer de la mémoire en masse <> loadData()
#del df4;
debug;

Countries Having no correct nutriments data at all : 

Bangladesh,	Dominican Republic,	Panama,	Trinidad and Tobago,	Barbados,	Haiti,	El Salvador,	Guatemala,	Oman,	United-states-of-america,	Guinea,	Jordan,	Ecuador,	Puerto Rico,	Nigeria,	Honduras,	Monaco,	Mongolia,	Myanmar,	Bahrain,	Cameroon,	Other-العراق,	Global-market,	Schweiz,	Malawi,	Gabon,	Saint Martin,	Republic of the Congo,	Malaysia,	Mayotte,	Latvia,	Wallis and Futuna,	Niger,	Republic of Macedonia,	Bosnia and Herzegovina,	Belarus,	Mauritania,	Iran,	Cyprus,	Afghanistan,	Mali,	日本,	Chad,	Estonia,	Sri Lanka,	Other-日本,	Libya,	Jersey,	Burundi,	Democratic Republic of the Congo,	Ghana,	Guinea-Bissau,	Bolivia,	Central African Republic,	Angola,	البحرين,	Georgia,	Namibia,	Mozambique,	Tanzania,	Rwanda,	Brunei,	Somalia,	Palestinian territories,	Vanuatu,	Yemen,	Ap,	Isle of Man,	Uruguay,	Paraguay,	Quebec,	Ethiopia,	Cook Islands,	Fiji,	Maldives
Nb of countries with correct nutriments data : 1241
nutriments boundaries processed


### Pays vagues
Certains pays sont vaguement définis, ou sont des catégories, ou des continents .. non pertinents ..
- OR are some kind of categories.. 
- never learnt to read arabic or chinese and some countries mapping defined in each respective language are even impossible to map accordingly .. 
- ISO COUNTRY CODE ARE MADE THIS FOR !!!! =>Republique-de-chine

In [ ]:
drop_countries = ['European Union', 'Other-japon', 'Other-turquie','World','fr:Quebec', 'الإمارات-العربية-المتحدة' ,'السعودية' ,'تونس', 'سلطنة-عمان', 'لبنان', '香港','en:fresh-foods','en:dairies']
keys=country2nutriments.keys();
for i in drop_countries:
    if i in keys:
        del country2nutriments[i]
#country2nutriments = country2nutriments[np.logical_not(country2nutriments['country'].isin(drop_countries))]    
# Display the cleaned dataset
dict(list(country2nutriments.items())[:2])

{'France': {'alcohol': {'ko': 0,
   'mean': 11.83,
   'ok': 2086,
   'oob': 0,
   'per': 1.0},
  'carbohydrates': {'ko': 0, 'mean': 28.78, 'ok': 72641, 'oob': 0, 'per': 1.0},
  'carbon-footprint': {'ko': 0,
   'mean': 368.03,
   'ok': 166,
   'oob': 0,
   'per': 1.0},
  'cholesterol': {'ko': 0, 'mean': 0.07, 'ok': 104, 'oob': 0, 'per': 1.0},
  'energy': {'ko': 0, 'mean': 1117.44, 'ok': 91416, 'oob': 0, 'per': 1.0},
  'fat': {'ko': 0, 'mean': 14.58, 'ok': 69835, 'oob': 0, 'per': 1.0},
  'fiber': {'ko': 0, 'mean': 3.73, 'ok': 43553, 'oob': 0, 'per': 1.0},
  'omega-3-fat': {'ko': 0, 'mean': 3.17, 'ok': 697, 'oob': 0, 'per': 1.0},
  'proteins': {'ko': 0, 'mean': 8.4, 'ok': 85674, 'oob': 0, 'per': 1.0},
  'salt': {'ko': 0, 'mean': 1.3, 'ok': 81188, 'oob': 0, 'per': 1.0},
  'sugars': {'ko': 0, 'mean': 14.32, 'ok': 83649, 'oob': 0, 'per': 1.0}},
 'United States': {'alcohol': {'ko': 0,
   'mean': 12.83,
   'ok': 46,
   'oob': 0,
   'per': 1.0},
  'carbohydrates': {'ko': 0,
   'mean': 37.83,
  

###  Dataframe valeurs nutrionitionnelles par pays
Création d'un nouveau dataFrame à partir du dictionnaire de valeurs nutritives moyennes par pays ..
- rempli 0 par valeur manquante

In [ ]:
#import json;
#}{cast dict to array once again : new Pandas DataFrame out of json data
cols='country,sum_ok'.split(',');
for nutriment in nutriments:
    cols=cols+[nutriment+'_mean',nutriment+'_ok',nutriment+'_ko',nutriment+'_per',nutriment+'_oob']

df3=pd.DataFrame(columns=cols)#pd.DataFrame'int' object is not iterable
countries=list(country2nutriments.keys());
cc=len(countries)

i=0;
for country in country2nutriments.keys():
    v=country2nutriments[country];
    sumok=0
    #print(country);print(v['sugars']['ok']);
    for j in nutriments:        
        if(j not in v or type(v[j])!=dict):#moldavian sugar is NAN
            #print('notice: empty values: '+country+' & '+j)
            v[j]={'mean':0,'ok':0,'ko':0,'per':0,'oob':0}
        sumok=sumok+v[j]['ok']
        
    data=[country,sumok]
    for j in nutriments:        
        data=data+[v[j]['mean'],v[j]['ok'],v[j]['ko'],v[j]['per'],v[j]['oob']]
        #,,v['proteins']['mean'],v['proteins']['ok'],v['proteins']['ko'],v['proteins']['per'],v['fat']['mean'],v['fat']['ok'],v['fat']['ko'],v['fat']['per']
    #print(v)
    df3.loc[i]=data
#filter out DATA
    i=i+1;
    #TypeError: unhashable type: 'list'
    
df3[:1]

,country,sum_ok,energy_mean,energy_ok,energy_ko,energy_per,energy_oob,fat_mean,fat_ok,fat_ko,fat_per,fat_oob,proteins_mean,proteins_ok,proteins_ko,proteins_per,proteins_oob,carbohydrates_mean,carbohydrates_ok,carbohydrates_ko,...,salt_mean,salt_ok,salt_ko,salt_per,salt_oob,alcohol_mean,alcohol_ok,alcohol_ko,alcohol_per,alcohol_oob,fiber_mean,fiber_ok,fiber_ko,fiber_per,fiber_oob,carbon-footprint_mean,carbon-footprint_ok,carbon-footprint_ko,carbon-footprint_per,carbon-footprint_oob
0,France,531009,1117.44,91416,0,1.0,0,14.58,69835,0,1.0,0,8.4,85674,0,1.0,0,28.78,72641,0,...,1.3,81188,0,1.0,0,11.83,2086,0,1.0,0,3.73,43553,0,1.0,0,368.03,166,0,1.0,0


### Graph top 10 par nutriment par pays
>> Limiter les données d'affichage à partir de 90 records ok pour chaque type de nutriments pour tente de jolis graphs

In [ ]:
#df4 = df3[(df3['sugars_ok'] > 30) & (df3['proteins_ok'] > 30) & (df3['fat_ok'] > 150)]
#https://towardsdatascience.com/a-guide-to-pandas-and-matplotlib-for-data-exploration-56fad95f951c
limiteValeursNecessaires=20
top30Countries=df3[(df3.country != 'na') & (df3['sum_ok']>90)].sort_values(by='sum_ok',ascending=False)[['country','sugars_mean','proteins_mean','fat_mean','energy_mean','salt_mean','fiber_mean']][:30];

plt.rcParams["figure.figsize"] = (32,32)#WIDE
x='190-bars-perCountry.png'
top30Countries.plot(kind='bar',x='country',subplots=True);plt.savefig(x,bbox_inches='tight');webp(x);show();#rot=0,,color=['#FF8C00','#FFD700','#FF4500','#FF0077']


#barh -> horizonale bar
#TOP 10 Nutriments Classified By Country, then per brandName ???
cols=3;rows=math.ceil(len(nutriments*3)/cols)
f,ax=plt.subplots(rows,cols,figsize=(24,8*rows))#total figsize
ax=ax.flatten();z=0

#ayant plus de 30 valeurs renseignées
for i in nutriments:#df['fiber_100g'].max()
  #print(i)
  #print('_'*60);top10.values
  #top10=df3[(df3.country != 'na') & (df4[i+'_ok']>30)].groupby('country')[i+'_ok',i+'_mean'].sum().sort_values(by=i+'_mean',ascending=False).head(10);
  top10=df3[(df3.country != 'na') & (df3[i+'_ok']>limiteValeursNecessaires)].sort_values(by=i+'_mean',ascending=False)[['country',i+'_mean']][:10];
  #i='energy-from-fat';df3[(df3.country != 'na') & (df3[i+'_ok']>30)].sort_values(by=i+'_mean',ascending=False)[['country',i+'_mean']][:10];#only us so
  if(top10.size < 3):#includes head :: #ValueError: `dataset` input should have multiple elements. for KDE graphe
    print('Not enough candidates for '+i+'..');
    continue;
#IndexError: index 12 is out of bounds for axis 0 with size 12  
  top10.plot(ax=ax[z],kind='bar',x='country');z+=1
  #top10.plot(ax=ax[1]);plt.title('Mean '+i+' per country');
  top10.plot(ax=ax[z],x='country');plt.title('Mean '+i+' per country');z+=1
  top10.plot(ax=ax[z],kind='kde');plt.title('Mean '+i+' repartition');z+=1;#Kernel Density Mean 80% SUgar is around 18% of top ten    
  #top10.plot(ax=ax[3],kind='pie',x=top10['country'],y=i+'_mean',shadow=True,autopct='%1.1f%%');plt.legend({i+' repartition'});#,explode=top10.values
  #print(top10);#TOP AVG}

plt.xticks(rotation=45)
plt.legend({i+' repartition'});

x='190-nutriments-par-pays.png'
f.savefig(x,bbox_inches='tight');
ftpput(x)    
show();

put:http://1.x24.fr/a/jupyter/off/190-bars-perCountry.png
put:http://1.x24.fr/a/jupyter/off/190-nutriments-par-pays.png


### Champions

Les champions sont : 
- Madagascar for carbohydrates ( no refined sugars from bread or fruits, cereals, seeds, vegetables  ), beware of how classified is the data, as there is many synonyms for sugar
- Russia is the Champion for Salt, Alcohol and Fiber contained within it's food ( rows ) : must be some Salted Vodkas containing Grass within
- France for Cholesterol ( must be a few rows within the dataset )
- Sweden for average Calories ( energy )
- Romania for proteins
- Poland for sugar
- Tunisia for fat

### Maleureusment :
- Il y a très peu de données concernant l'empreinte carbone ( uniquement en France et Suisse )

In [ ]:
# Write the data to a file, and persist it to the drive
df3.to_csv(fn3, sep='\t')#10ko only !!
os.system('zip '+fn3+'.zip '+fn3);
ftpput(fn3+'.zip')
#os.system('cp open_food_data_analysis.tsv.zip drive/My Drive/@docs/Data/open_food_data_analysis.tsv.zip');

put:http://1.x24.fr/a/jupyter/off/en.openfoodfacts.org.products.tsv3.tsv.zip


# eXperimentations !

In [ ]:
assert(False)

!pip install py-sphviewer
import numpy as np
import numpy.random
import matplotlib.pyplot as plt
import sphviewer as sph

def myplot(x, y, nb=32, xsize=500, ysize=500):   
    xmin = np.min(x)
    xmax = np.max(x)
    ymin = np.min(y)
    ymax = np.max(y)

    x0 = (xmin+xmax)/2.
    y0 = (ymin+ymax)/2.

    pos = np.zeros([3, len(x)])
    pos[0,:] = x
    pos[1,:] = y
    w = np.ones(len(x))

    P = sph.Particles(pos, w, nb=nb)
    S = sph.Scene(P)
    S.update_camera(r='infinity', x=x0, y=y0, z=0, 
                    xsize=xsize, ysize=ysize)
    R = sph.Render(S)
    R.set_logscale()
    img = R.get_image()
    extent = R.get_extent()
    for i, j in zip(xrange(4), [x0,x0,y0,y0]):
        extent[i] += j
    print(extent)
    return img, extent

fig = plt.figure(1, figsize=(10,10))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)


# Generate some test data
x = np.random.randn(1000)
y = np.random.randn(1000)

#Plotting a regular scatter plot
ax1.plot(x,y,'k.', markersize=5)
ax1.set_xlim(-3,3)
ax1.set_ylim(-3,3)
#myplot ValueError: Data and query points must have same dimensions
heatmap_16, extent_16 = myplot(x,y, nb=16)
heatmap_32, extent_32 = myplot(x,y, nb=32)
heatmap_64, extent_64 = myplot(x,y, nb=64)

ax2.imshow(heatmap_16, extent=extent_16, origin='lower', aspect='auto')
ax2.set_title("Smoothing over 16 neighbors")

ax3.imshow(heatmap_32, extent=extent_32, origin='lower', aspect='auto')
ax3.set_title("Smoothing over 32 neighbors")

#Make the heatmap using a smoothing over 64 neighbors
ax4.imshow(heatmap_64, extent=extent_64, origin='lower', aspect='auto')
ax4.set_title("Smoothing over 64 neighbors")

show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.ndimage.filters import gaussian_filter

# Generate some test data
x = np.random.randn(1000)
y = np.random.randn(1000)

sigmas = [0, 2, 4, 6, 12, 24, 48, 96, 128]
fig, axs = plt.subplots(2, 4)#multiplication autant que de sigmas !
axes=axs.flatten()#4 axes possibles
cols=zip(axes, sigmas)
for ax, s in cols:
  #print(ax)#axes subplots
  if s == 0:
    ax.plot(x, y, 'k.', markersize=5)
    ax.set_title("Scatter plot")
  else:
    img, extent = myplot(x, y, s)
    ax.imshow(img, extent=extent, origin='lower', cmap=cm.jet)
    ax.set_title("Smoothing with  $\sigma$ = %d" % s)

plt.savefig('eXperiment.png', bbox_inches = 'tight')
webp('eXperiment.png')
show()

## 2

In [ ]:
#df=cleanData(loadData(df),0);df['carbohydrates_100g'].describe();df['carbohydrates_100g'].max()
import matplotlib;
plt.rcParams["figure.figsize"] = (22,6)#with left tab


plt.scatter(df['sugars_100g'],df['carbohydrates_100g'],edgecolors='none',s=20,c=df['carbohydrates_100g']);
#plt.scatter(df['sugars_100g'],df['carbohydrates_100g'],edgecolors='none',s=15,c=np.log(df['carbohydrates_100g']),norm=matplotlib.colors.Normalize());
plt.colorbar();show()#LogNorm,
#otherwise .. will be bugged with new data




getFile(fn2)
df=loadData(fn2);
df=df.drop(['Unnamed: 0'],axis=1)

#Le résultat souhaité
fig,ax=plt.subplots();fig.patch.set_facecolor('white');ax.set_xlim(0,100);ax.set_ylim(0,100);plt.scatter(df['sugars_100g'],df['carbohydrates_100g'],edgecolors='none',s=1,c='purple');plt.colorbar();show();#joli on appercoit des lignes se tracer ..


#!pip install pydataset;#ambiguous truth value
#from pydataset import data;q=data(df)



heat('sugars_100g','carbohydrates_100g')

plt.title("Smoothing with  $\sigma$ = %d" % s)

assert(False)


#https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/
%who str
%who int
%who dict
%who list


fn	 fn2	 fn3	 fn4	 fn5	 


## 3

In [ ]:
#ftpput('off-AllCorrelations.tsv')#z='off-AllCorrelations.tsv';getFile(z);x=pd.read_csv(z,sep='\t')#index is unamed :0 -> to_csv(index=False)
plt.rcParams["figure.figsize"] = (6,6)

assert(False)

import math


correlations=fgc('off-AllCorrelations')
top30=correlations.where(np.triu(np.ones(correlations.shape),k=1).astype(np.bool)).stack().sort_values(ascending=False)[:30]
idx=top30.index;#carbonara
taille=len(top30)
#math.ceil(taille/6)*2

#repartir le top30
top30=correlations.where(np.triu(np.ones(correlations.shape),k=1).astype(np.bool)).stack().sort_values(ascending=False)[:30]
idx=top30.index;#carbonara

j=0;axx=6;axy=10;
f,axes=plt.subplots(axy,axx,figsize=(axy*6,axx*6))
f.patch.set_facecolor('white')
axes=axes.flatten()

j=0;
for i in range(len(top30)):  
  x=idx[i][0];
  y=idx[i][1];
  #maxx=df[x].max();maxy=df[y].max();if maxy>maxx: maxx=maxy
  heat(df,x,y,2,axis=axes[j])
  j=j+1;
  scatter(df,x,y,opacity=0.002,color='purple',size=5,axis=axes[j])
  j=j+1;

f.savefig('big.png', bbox_inches = 'tight')
webp('big.png')  
show()

  if(j>axx-1):
    j=0;k=k+1;
  
  continue;
  #print(y+'/'+x)
  print(i)
  print(type(idx[i]))#tuple
  print(idx[i])
  print(top30[i])
  print('_'*160)

f.savefig('big.png')
webp('big.png')  
show()

scatter(df,'energy_100g','fat_100g',opacity=0.002,color='purple',size=5)
heat(df,'energy_100g','fat_100g',1,0,100,100)
img


## 4 - Contour


In [ ]:
#python 3 variables into contour plot
def fmt(x, y):
  # get closest point with known data
  dist = np.linalg.norm(np.vstack([Xflat - x, Yflat - y]), axis=0)
  idx = np.argmin(dist)
  z = Zflat[idx]
  return 'x={x:.5f}  y={y:.5f}  z={z:.5f}'.format(x=x, y=y, z=z)

def contour(data,x,y,z,fn=0):
  from scipy.interpolate import griddata
  df=data.copy(deep=True)#&
  if(fn==0):
    fn='3.contour-'+z+'-'+x+'-'+y+'-'+str(levels)+'-'+str(limit)+'.png'
  #data Cleanup
  for i in [x,y,z]:
    if(df.dtypes[i] != object):#integer values only
      df.loc[np.isnan(df[i]),i]=0
      df.loc[(df[i]<0),i]=0 
      df.loc[(df[i]>getLimit(i)),i]=getLimit(i)

  x0=df[x];y0=df[y];z0=df[z]
  plt.title(z+' / '+x+' & '+y);plt.xlabel(x);plt.ylabel(y)   
#CS=plt.tricontour(x0,y0,z0,levels)
  x1 = np.linspace(x0.min(), x0.max(), len(x0.unique()))#pour autant de valeurs uniques, espacer et créer le contour
  y1 = np.linspace(y0.min(), y0.max(), len(y0.unique()))
  x2, y2 = np.meshgrid(x1, y1)
  #z2 = griddata(x, y, z, xi, yi, interp='linear')
  z2 = griddata((x0, y0), z0, (x2, y2), method='linear')#, method='cubic' => throws out of bound values
  #Xflat, Yflat, Zflat = x2.flatten(), y2.flatten(), z2.flatten()
  #plt.contourf(x2,y2,z2)
  #plt.zlabel('energy_100g')
  #fig = plt.figure()
  #ax = fig.gca()
  #ax.set_zlim(0,getLimit(z))
  CS = plt.contourf(x2, y2, z2,levels, vmin=0, vmax=getLimit(z))#First,zlim=getLimit(z)
  #plt.clim(0,getLimit(z))#otherwise strange negative value will show up
  #plt.scatter(x0,y0,alpha=200/limit,s=2)
  CS2 = plt.contour(CS, levels=CS.levels[::2],colors='black',alpha=0.1)#contour tous les 2 niveaux, mais si ils sont trop densément rapprochés à certains endroits .. ??
  cbar = plt.colorbar(CS);cbar.ax.set_ylabel(z);cbar.add_lines(CS2);#Legend
  plt.tight_layout()
  plt.savefig(fn, bbox_inches='tight')
  webp(fn) 
  show()

x='fat_100g';y='carbohydrates_100g';z='energy_100g';
levels=20;
limit=400000;#plantage not enough ram sinon .. exclure les valeurs pourries
heightPerGraph=widthPerGraph=24
plt.rcParams["figure.figsize"] = (widthPerGraph,heightPerGraph)
#Filtrer les résultats abérrants
# & (df['energy_100g']<3900) & (df['energy_100g']>0)
dfLimited=df[(df['sum']<100)][:limit]

contour(dfLimited,x,y,z,fn)  
#otherwise strange negative value will show up
#plt.gca().format_coord = fmt#aucun effet

# Add the contour line levels to the colorbar
#plt.contourf(x2,y2,z2,levels=20)#range(int(z.max())))#[0,500,1000,1500,2000,2500,3000,3500,4000])
assert(False)

from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d, Axes3D #<-- Note the capitalization! 
fig = plt.figure()
ax = Axes3D(fig) 
#ax = fig.gca(projection='3d')#Unknown projection '3d'
surf = ax.plot_surface(x2, y2, z2, rstride=1, cstride=1, cmap=cm.coolwarm,linewidth=0, antialiased=False)
#ax.set_zlim(-1.01, 1.01)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.title('Meshgrid Created from 3 1D Arrays')
show()

#https://stackoverflow.com/questions/9170838/surface-plots-in-matplotlib/30539444#30539444
#££££!!https://stackoverflow.com/questions/43205289/plotting-a-contour-plot-of-categorical-values-using-matplotlib

##5 - Radar : usefull for single record
- https://plot.ly/python/radar-chart/

In [ ]:
def radar(sel,seuilSuppression=1):
  if('energy_100g' in sel.keys()):
    sel['energy_100g']=sel['energy_100g']/370
  #print(sel);print(sel['product_name'].values[0]);print(sel['code'].values[0])
  title=sel['product_name'].values[0]+' - '+str(sel['code'].values[0])

  for i in sel.keys():
    #print(i);print(sel[i].values[0])
    if(i in 'code,product_name'.split(',')):
      del(sel[i]);
    elif(sel[i].values[0]<seuilSuppression):
      del(sel[i]);
      #print('del:'+i)
#Exception: Data must be 1-dimensional
  radardf = pd.DataFrame(dict(  r=sel.values[0], theta=sel.columns))
  fig = px.line_polar(radardf, r='r', theta='theta', line_close=True,title=title)
  fig.update_traces(fill='toself')
  fig.show()

cols='carbohydrates_100g,proteins_100g,fat_100g,sugars_100g,energy_100g'.split(',')
cols=Cols100g.copy()
cols+='code,product_name'.split(',')
product=dfQuery(df4,'code','16193')
#product=dfQuery(df4,'code','3065620011192')
radar(product[cols],2)  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## 6 - Radar

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection


def radar_factory(num_vars, frame='circle'):
    """Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle' | 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    # rotate theta such that the first axis is at the top
    theta += np.pi/2

    def draw_poly_patch(self):
        verts = unit_poly_verts(theta)
        return plt.Polygon(verts, closed=True, edgecolor='k')

    def draw_circle_patch(self):
        # unit circle centered on (0.5, 0.5)
        return plt.Circle((0.5, 0.5), 0.5)

    patch_dict = {'polygon': draw_poly_patch, 'circle': draw_circle_patch}
    if frame not in patch_dict:
        raise ValueError('unknown value for `frame`: %s' % frame)

    class RadarAxes(PolarAxes):

        name = 'radar'
        # use 1 line segment to connect specified points
        RESOLUTION = 1
        # define draw_frame method
        draw_patch = patch_dict[frame]

        def fill(self, *args, **kwargs):
            """Override fill so that line is closed by default"""
            closed = kwargs.pop('closed', True)
            return super(RadarAxes, self).fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super(RadarAxes, self).plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.concatenate((x, [x[0]]))
                y = np.concatenate((y, [y[0]]))
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            return self.draw_patch()

        def _gen_axes_spines(self):
            if frame == 'circle':
                return PolarAxes._gen_axes_spines(self)
            # The following is a hack to get the spines (i.e. the axes frame)
            # to draw correctly for a polygon frame.

            # spine_type must be 'left', 'right', 'top', 'bottom', or `circle`.
            spine_type = 'circle'
            verts = unit_poly_verts(theta)
            # close off polygon by repeating first vertex
            verts.append(verts[0])
            path = Path(verts)

            spine = Spine(self, spine_type, path)
            spine.set_transform(self.transAxes)
            return {'polar': spine}

    register_projection(RadarAxes)
    return theta


def unit_poly_verts(theta):
    """Return vertices of polygon for subplot axes.

    This polygon is circumscribed by a unit circle centered at (0.5, 0.5)
    """
    x0, y0, r = [0.5] * 3
    verts = [(r*np.cos(t) + x0, r*np.sin(t) + y0) for t in theta]
    return verts


def example_data():
    data = [
        ['Sulfate', 'Nitrate', 'EC', 'OC1', 'OC2', 'OC3', 'OP', 'CO', 'O3'],
        ('Basecase', [
            [0.88, 0.01, 0.03, 0.03, 0.00, 0.06, 0.01, 0.00, 0.00],
            [0.07, 0.95, 0.04, 0.05, 0.00, 0.02, 0.01, 0.00, 0.00],
            [0.01, 0.02, 0.85, 0.19, 0.05, 0.10, 0.00, 0.00, 0.00],
            [0.02, 0.01, 0.07, 0.01, 0.21, 0.12, 0.98, 0.00, 0.00],
            [0.01, 0.01, 0.02, 0.71, 0.74, 0.70, 0.00, 0.00, 0.00]]),
        ('With CO', [
            [0.88, 0.02, 0.02, 0.02, 0.00, 0.05, 0.00, 0.05, 0.00],
            [0.08, 0.94, 0.04, 0.02, 0.00, 0.01, 0.12, 0.04, 0.00],
            [0.01, 0.01, 0.79, 0.10, 0.00, 0.05, 0.00, 0.31, 0.00],
            [0.00, 0.02, 0.03, 0.38, 0.31, 0.31, 0.00, 0.59, 0.00],
            [0.02, 0.02, 0.11, 0.47, 0.69, 0.58, 0.88, 0.00, 0.00]]),
        ('With O3', [
            [0.89, 0.01, 0.07, 0.00, 0.00, 0.05, 0.00, 0.00, 0.03],
            [0.07, 0.95, 0.05, 0.04, 0.00, 0.02, 0.12, 0.00, 0.00],
            [0.01, 0.02, 0.86, 0.27, 0.16, 0.19, 0.00, 0.00, 0.00],
            [0.01, 0.03, 0.00, 0.32, 0.29, 0.27, 0.00, 0.00, 0.95],
            [0.02, 0.00, 0.03, 0.37, 0.56, 0.47, 0.87, 0.00, 0.00]]),
        ('CO & O3', [
            [0.87, 0.01, 0.08, 0.00, 0.00, 0.04, 0.00, 0.00, 0.01],
            [0.09, 0.95, 0.02, 0.03, 0.00, 0.01, 0.13, 0.06, 0.00],
            [0.01, 0.02, 0.71, 0.24, 0.13, 0.16, 0.00, 0.50, 0.00],
            [0.01, 0.03, 0.00, 0.28, 0.24, 0.23, 0.00, 0.44, 0.88],
            [0.02, 0.00, 0.18, 0.45, 0.64, 0.55, 0.86, 0.00, 0.16]])
    ]
    return data


if __name__ == '__main__':
    N = 9
    theta = radar_factory(N, frame='polygon')

    data = example_data()
    spoke_labels = data.pop(0)

    fig, axes = plt.subplots(figsize=(9, 9), nrows=2, ncols=2,
                             subplot_kw=dict(projection='radar'))
    fig.subplots_adjust(wspace=0.25, hspace=0.20, top=0.85, bottom=0.05)

    colors = ['b', 'r', 'g', 'm', 'y']
    # Plot the four cases from the example data on separate axes
    for ax, (title, case_data) in zip(axes.flatten(), data):
        ax.set_rgrids([0.2, 0.4, 0.6, 0.8])
        ax.set_title(title, weight='bold', size='medium', position=(0.5, 1.1),
                     horizontalalignment='center', verticalalignment='center')
        for d, color in zip(case_data, colors):
            ax.plot(theta, d, color=color)
            ax.fill(theta, d, facecolor=color, alpha=0.25)
        ax.set_varlabels(spoke_labels)

    # add legend relative to top-left plot
    ax = axes[0, 0]
    labels = ('Factor 1', 'Factor 2', 'Factor 3', 'Factor 4', 'Factor 5')
    legend = ax.legend(labels, loc=(0.9, .95),
                       labelspacing=0.1, fontsize='small')

    fig.text(0.5, 0.965, '5-Factor Solution Profiles Across Four Scenarios',
             horizontalalignment='center', color='black', weight='bold',
             size='large')

    show()

## 7 - WordCloud
- http://1.x24.fr/a/jupyter/off/wordcloud.product_name.png

In [ ]:
#resume()
os.system('git clone https://github.com/amueller/word_cloud.git;cd word_cloud;pip install .')
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def wordCloud(df,k):
  stopwords_en = set(stopwords.words('english'))
  bigText=''
#wordCloud(df,'product_name')
  x='wordcloud.'+k+'.png'
  #bigText = \" \".join(txt for txt in df['countries_en'])
  for i in df[k]:
    if(type(i)!=str):#escape 0 NaN potential values
      #print(str(i)+':'+str(type(i)));
      continue;
    bigText+=' '+re.sub(r"[^a-z0-9:]+",'', i.lower())

  print ("There are {} words in the combination of all ".format(len(bigText)))
  # Generate a word cloud image
  wordcloud = WordCloud(stopwords=stopwords_en,background_color="white").generate(bigText)#
  # Display the generated image:
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis("off")
  plt.savefig(x, bbox_inches='tight')
  webp(x) 
  show()

plt.rcParams["figure.figsize"] = (24,24)
wordCloud(df,'product_name')
#wordCloud(df,'countries_en')

## 8 -  ?

In [ ]:
%%script false 

from matplotlib.pyplot import cm

def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)

#return opaque color
def _c(x,o=0):
  if(type(x[0])==np.ndarray):
    x[0][3]=1
    #print(x[0])
    return x[0]
  elif o:
    x[3]=o
    #print(x)
    return [x];        

#_c(p1.get_facecolors())
def showLabelsOnly(df,i,yCols,xlabel="100g"):
  j=yCols.split(',')
  firstCol=j.pop(0)
  markers='s;^;o;v;D;>;<;1;2;3;4;8;.;,;p;P;*;h;H;+;x;X;d;|;_'.split(';')
  plt.rcParams["figure.figsize"] = (2,1)
  colors=cm.rainbow(np.linspace(0,1,len(j)+1))#iter(list)
  yaxises={};scatplots={};cn=0;dec=1;
  fig,yaxises[firstCol] = plt.subplots()#host=par1#.figure(à,.gca()=get currecny axis)
##
  fig.subplots_adjust(right=0.75)
  for k in j:
    yaxises[k]=yaxises[firstCol].twinx()
    yaxises[firstCol].set_xlim(-100,-99)
    yaxises[k].set_ylim(-100,-99)
  for k in yaxises.keys():
    scatplots[k]=yaxises[k].scatter(df[k], df[i], label=k, c=_c(colors[cn], 1),marker=markers[cn]);cn=cn+1;

  yaxises[firstCol].set_xlim(-100,-99)
  yaxises[firstCol].set_ylim(-100,-99)  
  yaxises[firstCol].set_ylabel(i+'/'+firstCol);
  yaxises[firstCol].set_xlabel(xlabel);

  lines=list(scatplots.values())
  labels=[l.get_label() for l in lines]
  leg=yaxises[firstCol].legend(lines,labels,loc=1)
  leg.get_frame().set_facecolor('white')
  leg.get_frame().set_alpha(1)
  show()

def multipleScatter(df,i,yCols,xmin=0,xmax=0,ymin=0,ymax=0,xlabel="100g",size=15,opacity=0.02,column2alpha={},column2size={}):
  j=yCols.split(',')
  firstCol=j.pop(0);
#https://matplotlib.org/3.1.1/api/markers_api.html  
  markers='s;^;o;v;D;>;<;1;2;3;4;8;.;,;p;P;*;h;H;+;x;X;d;|;_'.split(';')
  colors=cm.rainbow(np.linspace(0,1,len(j)+1))#iter(list)
  yaxises={};scatplots={};dec=1;cn=0;
  fig,yaxises[firstCol]=plt.subplots()#host=par1#.figure(à,.gca()=get currecny axis)
  fig.subplots_adjust(right=0.75)
  #print(firstCol);print('j is:');print(j)
  #foreach duplicated yaxis
  for k in j:
    #print(firstCol+' cloned to : '+k)
    yaxises[k]=yaxises[firstCol].twinx()#Replicate First one for as much columns
    if(xmax):
      yaxises[k].set_xlim(xmin,xmax)
    if(ymax):
      yaxises[k].set_ylim(ymin,ymax)
    yaxises[k].set_ylabel(i+'/'+k);
    yaxises[k].spines["right"].set_position(("axes",dec))#décalage
    dec=dec+0.09
    #dec=dec+0.05;yaxises[k].spines["right"].set_position(("axes",dec))
  #ax1.set_zorder(1) 
  if"xsolo":
    if(xmax):
      yaxises[firstCol].set_xlim(xmin,xmax)
    if(ymax):
      yaxises[firstCol].set_ylim(ymin,ymax)
    yaxises[firstCol].set_ylabel(i+'/'+firstCol);
    yaxises[firstCol].set_xlabel(xlabel);

  for k in yaxises.keys():
    #print('_'*150);print(k);print(df[k][:1]);print(yaxises[k]);   print(colors[cn]);  print(markers[cn]);   
    #IndexError: index 3 is out of bounds for axis 0 with size 3
    if(k in column2alpha.keys()):
      o=column2alpha[k]
    else:
      o=opacity;
    if(k in column2size.keys()):
      s=column2size[k]
    else:
      s=size;
    a=df[k];b=df[i];c=_c(colors[cn],opacity);m=markers[cn]
    scatplots[k]=yaxises[k].scatter(a,b, label=k, c=c, s=s,marker=m);
    cn=cn+1;#+ Affectation des couleurs pour toutes, première comprise
    
  lines=list(scatplots.values())
  labels=[l.get_label() for l in lines]
  leg=yaxises[firstCol].legend(lines,labels,loc=1)#=plt.legendregroupée en une seule légende, facecolor is background ...
  #dir(leg)  #get_methods leg.axes.legend().get_lines()
  if False:
    #scatplots[firstCol]=yaxises[firstCol].scatter(df[firstCol], df[i], label=firstCol, c=_c(colors[cn],0.02),alpha=0.02);
    #h,l=yaxises[firstCol].get_legend_handles_labels()
    leg.set_alpha(1)
    for l in leg.get_lines():#is empty .. why ???
      #l.set_facecolor=
      l.set_alpha(1.0)

  # Offset the right spine of par2.  The ticks and label have already been placed on the right by twinx above.
  # Having been created by twinx, par2 has its frame off, so the line of its detached spine is invisible.  First, activate the frame but make the patch and spines invisible.
  # Second, show the right spine.
  par2=yaxises[j[0]];
  make_patch_spines_invisible(par2)
  par2.spines["right"].set_visible(True)

  x='multifactorial.'+yCols+'.png'
  #<Figure size 1440x1440 with 3 Axes>,TypeError: unhashable type: 'numpy.ndarray'
  fig.savefig(x, bbox_inches = 'tight') 
  webp(x)
  show()  
##
i='energy_100g'
#yCols='fat_100g,proteins_100g,carbohydrates_100g,sugars_100g,salt_100g,fiber_100g,saturated-fat_100g'
yCols='fat_100g,carbohydrates_100g,proteins_100g'

##
plt.rcParams["figure.figsize"] = (2,1)
showLabelsOnly(df,i,yCols)
##
xmin=0;ymin=0;xmax=100;ymax=3700
plt.rcParams["figure.figsize"] = (12,12)
multipleScatter(df,i,yCols,xmin,xmax,ymin,ymax,column2alpha={'proteins_100g':0.2},column2size={'proteins_100g':30})

def scatter3d(df,x,y,z,o=0.02):
  from mpl_toolkits.mplot3d import Axes3D  
  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')
  #dir(ax)
  ax.set_xlim(100)
  ax.set_ylim(100)
  ax.set_zlim(3700)
  ax.set_xlabel(x, fontsize=20)
  ax.set_ylabel(y, fontsize=20)
  ax.set_zlabel(z, fontsize=20)
  #ax.set_xticks(labels)
  #ax.xaxis.set_ticklabels(labels);#<= catégories unoiquement
  #3d ax OverflowError: Exceeded cell block limit (set 'agg.path.chunksize' rcparam)
  #matplotlib.rcParams['agg.path.chunksize'] = 10000
  #ax.plot(xs=df['fat_100g'], ys=df['carbohydrates_100g'],zs=df[i])
  ax.scatter(xs=df[x], ys=df[y],zs=df[z],alpha=o)
  
  ax.elev+=1;ax.azim+=1
  #for ii in xrange(0,360,1):
    #ax.view_init(elev=10., azim=ii)
  show()

plt.rcParams["figure.figsize"] = (12,12)
scatter3d(df,'fat_100g','carbohydrates_100g','energy_100g')



"""
assert(False)

for xe, ye in zip(j, i):
  plt.scatter([xe] * len(ye), ye)
  show()

plt.scatter(df[j],df[i])
https://plot.ly/python/3d-axes/
https://stackoverflow.com/questions/23698850/manually-set-color-of-points-in-legend

amounts=GaussianNB, ( not Classifier )
cv=#stratify => output categories
Pour utiliser l’intégralité de nos données pour entraîner et pour tester, et pour éviter un biais potentiel lié au fait de faire une évaluation unique, on préfère faire une validation croisée.
Dans le cas d’un problème de classification, on fait attention à stratifier la validation croisée pour éviter d’introduire des biais.
"""